In [2]:
import os
import json
from pprint import pprint

import torch
import torch.nn as nn

from datasets import load_dataset, DatasetBuilder

import transformers
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)

from huggingface_hub import notebook_login

In [2]:
notebook_login()

# Set running device


In [3]:
{
    "cells": [
        {
            "cell_type": "code",
            "execution_count": 2,
            "metadata": {},
            "outputs": [],
            "source": [
                "import os\n",
                "import json\n",
                "from pprint import pprint\n",
                "\n",
                "import torch\n",
                "import torch.nn as nn\n",
                "\n",
                "from datasets import load_dataset, DatasetBuilder\n",
                "\n",
                "import transformers\n",
                "from transformers import (\n",
                "    AutoConfig,\n",
                "    AutoModelForCausalLM,\n",
                "    AutoTokenizer,\n",
                "    BitsAndBytesConfig,\n",
                ")\n",
                "\n",
                "from peft import (\n",
                "    LoraConfig,\n",
                "    PeftConfig,\n",
                "    PeftModel,\n",
                "    get_peft_model,\n",
                "    prepare_model_for_kbit_training,\n",
                ")\n",
                "\n",
                "from huggingface_hub import notebook_login",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 2,
            "metadata": {},
            "outputs": [
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "12826930970c474a8c7b0feac6339b90",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "VBox(children=(HTML(value='<center> <img\\nsrc=https://huggingface.co/front/assets/huggingface_logo-noborder.sv…"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                }
            ],
            "source": ["notebook_login()"],
        },
        {"cell_type": "markdown", "metadata": {}, "source": ["# Set running device\n"]},
        {
            "cell_type": "code",
            "execution_count": 3,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": ["[INFO] Currently running device: cuda\n"],
                }
            ],
            "source": [
                "{\n",
                '    "cells": [\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 2,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "import os\\n",\n',
                '                "import json\\n",\n',
                '                "from pprint import pprint\\n",\n',
                '                "\\n",\n',
                '                "import torch\\n",\n',
                '                "import torch.nn as nn\\n",\n',
                '                "\\n",\n',
                '                "from datasets import load_dataset, DatasetBuilder\\n",\n',
                '                "\\n",\n',
                '                "import transformers\\n",\n',
                '                "from transformers import (\\n",\n',
                '                "    AutoConfig,\\n",\n',
                '                "    AutoModelForCausalLM,\\n",\n',
                '                "    AutoTokenizer,\\n",\n',
                '                "    BitsAndBytesConfig,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "from peft import (\\n",\n',
                '                "    LoraConfig,\\n",\n',
                '                "    PeftConfig,\\n",\n',
                '                "    PeftModel,\\n",\n',
                '                "    get_peft_model,\\n",\n',
                '                "    prepare_model_for_kbit_training,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "from huggingface_hub import notebook_login",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 2,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "12826930970c474a8c7b0feac6339b90",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "VBox(children=(HTML(value=\'<center> <img\\\\nsrc=https://huggingface.co/front/assets/huggingface_logo-noborder.sv…"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                }\n",
                "            ],\n",
                '            "source": ["notebook_login()"],\n',
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Set running device\\n"]},\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 3,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": ["[INFO] Currently running device: cuda\\n"],\n',
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                \'device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\\n\',\n',
                '                \'p   rint(f"[INFO] Currently running device: {device}")                                                               "import os\\\\n",\\n\',\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                        \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                              ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                              ,\\n",\n',
                "                \"                                          '',\\n\",\n",
                "                \"                '',\\n\",\n",
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                              \'\\"\\\\n\',\\n",\n',
                '                "                \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                                \'\\",\\\\n\',\\n",\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "                                                ,\\n",\n',
                '                "         ,\\n",\n',
                '                \'                                                                                 \\\'    """""\\\\n\\\',\\n\',\n',
                "                '                                                                                    \"\\\\n\",\\n',\n",
                '                "                        \'\\"\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                              ,\\n",\n',
                '                "                                                                                 ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                                                                                          \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                                    \'\\"none\\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                "                '                        \"\\\\n\",\\n',\n",
                '                "                        ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                                            \'\\"base\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                                                             ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                         \'\\\\\\"\',\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                "                '                                                                                                      \"\\\\n\",\\n',\n",
                '                "                                    \'\\"and\\", \',\\n",\n',
                "                '                                                                                          \"\\\\n\",\\n',\n",
                "                '                                                                                          \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                                        \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                        \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\',\\n",\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                "                '                        \\'\"name\": \",\\\\n\\',\\n',\n",
                '                "                                    ,\\n",\n',
                '                "               ,\\n",\n',
                '                "                                                      \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                        \'\\"pt\\"\\\\n\',\\n",\n',
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                "                \"                                                                                                                                          '',\\n\",\n",
                "                \"                                    '',\\n\",\n",
                '                "                  ,\\n",\n',
                '                "               ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                                          \'    \\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                '                "                \'\\"data\\",\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                  ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "markdown",\n',
                '            "metadata": {},\n',
                '            "source": ["# Load model & Tokenizer\\n"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 5,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "9247f78f9c3949b1949207c721e3c698",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "bb47838913be4050892e98a3dff5e963",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "b1691fce8bf04f1eb054a8c82f173855",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Downloading shards:   0%|          | 0/3 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "c223b4be9917420ba66600995e0b9f67",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "11a5e85988c84a7783ce746323bb83b4",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "e09398773fc146ea8ecede8f329bd954",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "8a929f80d6394bc3a1090fbfa36484b7",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "9d21a25ab6e64231ac4593e3078eefed",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "223aed03a73a4ddc85ff82f099475564",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "f1041d3c6e86420bb9a017620b8d34ec",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "659f749ddc7345ddb7ab3968f34965d8",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "fc4e61e6baad4335b1c69dc0998837cf",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "            ],\n",
                '            "source": [\n',
                "                'MODEL_NAME = \"mistralai/Mistral-7B-Instruct-v0.2\"\\n',\n",
                "                'weights_dir = \"./weights\"\\n',\n",
                '                "\\n",\n',
                '                "bnb_config = BitsAndBytesConfig(\\n",\n',
                '                "    load_in_4bit=True,\\n",\n',
                '                "    bnb_4bit_use_double_quant=True,\\n",\n',
                "                '    bnb_4bit_quant_type=\"nf4\",\\n',\n",
                '                "    bnb_4bit_compute_dtype=torch.bfloat16,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "model = AutoModelForCausalLM.from_pretrained(\\n",\n',
                '                "    MODEL_NAME,\\n",\n',
                '                "    device_map=device,\\n",\n',
                '                "    trust_remote_code=True,\\n",\n',
                '                "    quantization_config=bnb_config,\\n",\n',
                '                "    cache_dir=weights_dir,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "tokenizer = AutoTokenizer.from_pretrained(\\n",\n',
                '                "    MODEL_NAME, device=device, cache_dir=weights_dir\\n",\n',
                '                ")\\n",\n',
                '                "tokenizer.pad_token = tokenizer.eos_token",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 9,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "def print_trainable_parameters(model) -> None:\\n",\n',
                '                \'    """Prints the number of trainable parameters in the model."""\\n\',\n',
                '                "    trainable_params = 0\\n",\n',
                '                "    all_param = 0\\n",\n',
                '                "    for _, param in model.named_parameters():\\n",\n',
                '                "        all_param += param.numel()\\n",\n',
                '                "        if param.requires_grad:\\n",\n',
                '                "            trainable_params += param.numel()\\n",\n',
                '                "\\n",\n',
                '                "    print(\\n",\n',
                "                '        f\"Trainable params num: {trainable_params:,} perzentage: {100 * trainable_params / all_param} || All params: {all_param:,}\"\\n',\n",
                '                "    )",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 10,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "Trainable params num: 262,410,240 perzentage: 6.993743675173274 || All params: 3,752,071,168\\n"\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": ["print_trainable_parameters(model)"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 11,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "model.gradient_checkpointing_enable()\\n",\n',
                '                "model = prepare_model_for_kbit_training(model)",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 13,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "Trainable params num: 42,520,576 perzentage: 1.1205573318192514 || All params: 3,794,591,744\\n"\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                "config = LoraConfig(\\n",\n',
                '                "    r=16,\\n",\n',
                '                "    lora_alpha=32,\\n",\n',
                '                "    target_modules=[\\n",\n',
                "                '        \"q_proj\",\\n',\n",
                "                '        \"k_proj\",\\n',\n",
                "                '        \"v_proj\",\\n',\n",
                "                '        \"o_proj\",\\n',\n",
                "                '        \"gate_proj\",\\n',\n",
                "                '        \"up_proj\",\\n',\n",
                "                '        \"down_proj\",\\n',\n",
                "                '        \"lm_head\",\\n',\n",
                '                "    ],\\n",\n',
                '                "    lora_dropout=0.05,\\n",\n',
                "                '    bias=\"none\",\\n',\n",
                "                '    task_type=\"CAUSAL_LM\",\\n',\n",
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "model = get_peft_model(model, config)\\n",\n',
                '                "print_trainable_parameters(model)",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "markdown",\n',
                '            "metadata": {},\n',
                '            "source": ["# Test original model\\n"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 34,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                \'prompt_template_large = """\\n\',\n',
                '                "<s>[INST]\\n",\n',
                '                "# TASK DESCRIPTION\\n",\n',
                '                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                "\\n",\n',
                '                "# JSON PARAMETER VALUES\\n",\n',
                '                \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                "- input_name: Name of the input parameter for the function.\\n",\n',
                '                "- input_value: Value for the function input parameter.\\n",\n',
                '                "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                "\\n",\n',
                '                "# FUNCTION MEANING\\n",\n',
                '                "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                "\\n",\n',
                '                "# RESPONSE FORMAT\\n",\n',
                '                "- Include only necessary functions from the query.\\n",\n',
                '                "- Maintain the order of functions as in the query.\\n",\n',
                '                "- Note any uncertainties in case of ambiguous queries.\\n",\n',
                '                "\\n",\n',
                '                "# ADDITIONAL GUIDANCE\\n",\n',
                '                "- Focus on verbs and technical terms to determine the function.\\n",\n',
                '                "- Indicate if a query falls outside the available functions.\\n",\n',
                '                "- Consider practical aspects of robotic operations.\\n",\n',
                '                \'- Treat "base" as the first robot "joint".\\n\',\n',
                '                "\\n",\n',
                '                "# JSON FORMAT\\n",\n',
                '                "{{\\n",\n',
                "                '    \"functions\": [\\n',\n",
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                "                '            \"inputs\": [\\n',\n",
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\\n\',\n',
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\\n\',\n',
                '                "            ]\\n",\n',
                '                "        }},\\n",\n',
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                '                \'            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\\n\',\n',
                '                "        }}\\n",\n',
                '                "    ]\\n",\n',
                '                "}}\\n",\n',
                '                "\\n",\n',
                '                "USER QUERY: {user_query}\\n",\n',
                '                "[/INST]\\n",\n',
                '                \'""".strip()\',\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 51,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                \'prompt_template_small = """\\n\',\n',
                '                "<s>[INST]\\n",\n',
                '                "# TASK DESCRIPTION\\n",\n',
                '                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                "\\n",\n',
                '                "# JSON PARAMETER VALUES\\n",\n',
                '                \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                "- input_name: Name of the input parameter for the function.\\n",\n',
                '                "- input_value: Value for the function input parameter.\\n",\n',
                '                "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                "\\n",\n',
                '                "# FUNCTION MEANING\\n",\n',
                '                "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                "\\n",\n',
                '                "# JSON FORMAT\\n",\n',
                '                "{{\\n",\n',
                "                '    \"functions\": [\\n',\n",
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                "                '            \"inputs\": [\\n',\n",
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\\n\',\n',
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\\n\',\n',
                '                "            ]\\n",\n',
                '                "        }},\\n",\n',
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                '                \'            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\\n\',\n',
                '                "        }}\\n",\n',
                '                "    ]\\n",\n',
                '                "}}\\n",\n',
                '                "\\n",\n',
                '                "USER QUERY: {user_query}\\n",\n',
                '                "[/INST]\\n",\n',
                '                \'""".strip()\',\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 75,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "generation_config = model.generation_config\\n",\n',
                '                "g   eneration_config.max_new_tokens = 200\\n",\n',
                '                "ge      neration_config.temperature = 0.7\\n",\n',
                '                "gen         eration_config.top_p = 0.7\\n",\n',
                '                "gen         eration_config.num_return_sequences = 1\\n",\n',
                '                "         generation_config.pad_token_id = tokenizer.eos_token_id\\n",\n',
                "                'gen         eration_config.eos_token_id = tokenizer.eos_token_id                     \"import os\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                        \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                              ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                              ,\\n",\n',
                "                \"                                          '',\\n\",\n",
                "                \"                '',\\n\",\n",
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                              \'\\"\\\\n\',\\n",\n',
                '                "                \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                                \'\\",\\\\n\',\\n",\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "                                                ,\\n",\n',
                '                "         ,\\n",\n',
                '                \'                                                                                 \\\'    """""\\\\n\\\',\\n\',\n',
                "                '                                                                                    \"\\\\n\",\\n',\n",
                '                "                        \'\\"\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                              ,\\n",\n',
                '                "                                                                                 ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                                                                                          \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                                    \'\\"none\\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                "                '                        \"\\\\n\",\\n',\n",
                '                "                        ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                                            \'\\"base\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                                                             ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                         \'\\\\\\"\',\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                "                '                                                                                                      \"\\\\n\",\\n',\n",
                '                "                                    \'\\"and\\", \',\\n",\n',
                "                '                                                                                          \"\\\\n\",\\n',\n",
                "                '                                                                                          \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                                        \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                        \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\',\\n",\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                "                '                        \\'\"name\": \",\\\\n\\',\\n',\n",
                '                "                                    ,\\n",\n',
                '                "               ,\\n",\n',
                '                "                                                      \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                        \'\\"pt\\"\\\\n\',\\n",\n',
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                "                \"                                                                                                                                          '',\\n\",\n",
                "                \"                                    '',\\n\",\n",
                '                "                  ,\\n",\n',
                '                "               ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                                          \'    \\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                '                "                \'\\"data\\",\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                  ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 76,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\'user_query = "Obtain the status of the robot\\\'s third joint"\'],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 77,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "[INST]\\n",\n',
                '                        "# TASK DESCRIPTION\\n",\n',
                '                        "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                        "\\n",\n',
                '                        "# JSON PARAMETER VALUES\\n",\n',
                '                        \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                        "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                        "- input_name: Name of the input parameter for the function.\\n",\n',
                '                        "- input_value: Value for the function input parameter.\\n",\n',
                '                        "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                        "\\n",\n',
                '                        "# FUNCTION MEANING\\n",\n',
                '                        "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                        "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                        "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                        "\\n",\n',
                '                        "# JSON FORMAT\\n",\n',
                '                        "{\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "<name>",\\n\',\n',
                "                        '            \"inputs\": [\\n',\n",
                '                        \'                {"name": "<name>", "value": "<value>", "unit": "<unit>"},\\n\',\n',
                '                        \'                {"name": "<name>", "value": "<value>", "unit": "<unit>"}\\n\',\n',
                '                        "            ]\\n",\n',
                '                        "        },\\n",\n',
                '                        "        {\\n",\n',
                '                        \'            "function_name": "<name>",\\n\',\n',
                '                        \'            "inputs": [{"name": "<name>", "value": "<value>", "unit": "<unit>"}]\\n\',\n',
                '                        "        }\\n",\n',
                '                        "    ]\\n",\n',
                '                        "}\\n",\n',
                '                        "\\n",\n',
                '                        "USER QUERY: Obtain the status of the robot\'s third joint\\n",\n',
                '                        "[/INST] {\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "get_joint_values",\\n\',\n',
                "                        '            \"inputs\": []\\n',\n",
                '                        "        }\\n",\n',
                '                        "    ]\\n",\n',
                '                        "}\\n",\n',
                '                        "\\n",\n',
                '                        "USER QUERY: Move the robot\'s end effector to position (200, 300, 400) and orientation (0.1, 0.2, 0.3, 0.4)\\n",\n',
                '                        "{\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "move_tcp",\\n\',\n',
                "                        '            \"inputs\": [\\n',\n",
                '                        \'                {"name": "x", "value": 200, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "y", "value": 300, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "z", "value": 400, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "q1",\\n\',\n',
                '                        "CPU times: user 8.04 s, sys: 0 ns, total: 8.04 s\\n",\n',
                '                        "Wall time: 8.04 s\\n",\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                "%%time\\n",\n',
                "                'device = \"cuda:0\"\\n',\n",
                '                "\\n",\n',
                '                "encoding = tokenizer(\\n",\n',
                "                '    prompt_template_small.format(user_query=user_query), return_tensors=\"pt\"\\n',\n",
                '                ").to(device)\\n",\n',
                '                "with torch.inference_mode():\\n",\n',
                '                "    outputs = model.generate(\\n",\n',
                '                "        input_ids=encoding.input_ids,\\n",\n',
                '                "        attention_mask=encoding.attention_mask,\\n",\n',
                '                "        generation_config=generation_config,\\n",\n',
                '                "    )\\n",\n',
                '                "\\n",\n',
                '                "print(tokenizer.decode(outputs[0], skip_special_tokens=True))",\n',
                "            ],\n",
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Prepare data\\n"]},\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 5,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "799088593edf4f5ca4a9d96197c77670",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Downloading data files:   0%|          | 0/1 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "1624861f54584c94a26e6959e1efd8ff",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "7f613b42d76c4638a525ff4ec75f91ec",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Generating train split: 0 examples [00:00, ? examples/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "ename": "NotImplementedError",\n',
                '                    "evalue": "Loading a dataset cached in a LocalFileSystem is not supported.",\n',
                '                    "output_type": "error",\n',
                '                    "traceback": [\n',
                '                        "\\u001b[0;31m---------------------------------------------------------------------------\\u001b[0m",\n',
                '                        "\\u001b[0;31mNotImplementedError\\u001b[0m                       Traceback (most recent call last)",\n',
                '                        \'Cell \\u001b[0;32mIn[5], line 1\\u001b[0m\\n\\u001b[0;32m----> 1\\u001b[0m dataset \\u001b[38;5;241m=\\u001b[39m \\u001b[43mload_dataset\\u001b[49m\\u001b[43m(\\u001b[49m\\n\\u001b[1;32m      2\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43mjson\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      3\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[43mdata_files\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43m../DATA/processed_dataset_2024-Jan-30_23-29-10.json\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      4\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[43mfield\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43mdata\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      5\\u001b[0m \\u001b[43m)\\u001b[49m\\n\',\n',
                '                        "File \\u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/load.py:2149\\u001b[0m, in \\u001b[0;36mload_dataset\\u001b[0;34m(path, name, data_dir, data_files, split, cache_dir, features, download_config, download_mode, verification_mode, ignore_verifications, keep_in_memory, save_infos, revision, token, use_auth_token, task, streaming, num_proc, storage_options, **config_kwargs)\\u001b[0m\\n\\u001b[1;32m   2145\\u001b[0m \\u001b[38;5;66;03m# Build dataset for splits\\u001b[39;00m\\n\\u001b[1;32m   2146\\u001b[0m keep_in_memory \\u001b[38;5;241m=\\u001b[39m (\\n\\u001b[1;32m   2147\\u001b[0m     keep_in_memory \\u001b[38;5;28;01mif\\u001b[39;00m keep_in_memory \\u001b[38;5;129;01mis\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m \\u001b[38;5;28;01mNone\\u001b[39;00m \\u001b[38;5;28;01melse\\u001b[39;00m is_small_dataset(builder_instance\\u001b[38;5;241m.\\u001b[39minfo\\u001b[38;5;241m.\\u001b[39mdataset_size)\\n\\u001b[1;32m   2148\\u001b[0m )\\n\\u001b[0;32m-> 2149\\u001b[0m ds \\u001b[38;5;241m=\\u001b[39m \\u001b[43mbuilder_instance\\u001b[49m\\u001b[38;5;241;43m.\\u001b[39;49m\\u001b[43mas_dataset\\u001b[49m\\u001b[43m(\\u001b[49m\\u001b[43msplit\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43msplit\\u001b[49m\\u001b[43m,\\u001b[49m\\u001b[43m \\u001b[49m\\u001b[43mverification_mode\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43mverification_mode\\u001b[49m\\u001b[43m,\\u001b[49m\\u001b[43m \\u001b[49m\\u001b[43min_memory\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43mkeep_in_memory\\u001b[49m\\u001b[43m)\\u001b[49m\\n\\u001b[1;32m   2150\\u001b[0m \\u001b[38;5;66;03m# Rename and cast features to match task schema\\u001b[39;00m\\n\\u001b[1;32m   2151\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m task \\u001b[38;5;129;01mis\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m \\u001b[38;5;28;01mNone\\u001b[39;00m:\\n\\u001b[1;32m   2152\\u001b[0m     \\u001b[38;5;66;03m# To avoid issuing the same warning twice\\u001b[39;00m\\n",\n',
                '                        \'File \\u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/builder.py:1173\\u001b[0m, in \\u001b[0;36mDatasetBuilder.as_dataset\\u001b[0;34m(self, split, run_post_process, verification_mode, ignore_verifications, in_memory)\\u001b[0m\\n\\u001b[1;32m   1171\\u001b[0m is_local \\u001b[38;5;241m=\\u001b[39m \\u001b[38;5;129;01mnot\\u001b[39;00m is_remote_filesystem(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_fs)\\n\\u001b[1;32m   1172\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m is_local:\\n\\u001b[0;32m-> 1173\\u001b[0m     \\u001b[38;5;28;01mraise\\u001b[39;00m \\u001b[38;5;167;01mNotImplementedError\\u001b[39;00m(\\u001b[38;5;124mf\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mLoading a dataset cached in a \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mtype\\u001b[39m(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_fs)\\u001b[38;5;241m.\\u001b[39m\\u001b[38;5;18m__name__\\u001b[39m\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m is not supported.\\u001b[39m\\u001b[38;5;124m"\\u001b[39m)\\n\\u001b[1;32m   1174\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m os\\u001b[38;5;241m.\\u001b[39mpath\\u001b[38;5;241m.\\u001b[39mexists(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_output_dir):\\n\\u001b[1;32m   1175\\u001b[0m     \\u001b[38;5;28;01mraise\\u001b[39;00m \\u001b[38;5;167;01mFileNotFoundError\\u001b[39;00m(\\n\\u001b[1;32m   1176\\u001b[0m         \\u001b[38;5;124mf\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mDataset \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39mdataset_name\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m: could not find data in \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_output_dir\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m. Please make sure to call \\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1177\\u001b[0m         \\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mbuilder.download_and_prepare(), or use \\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1178\\u001b[0m         \\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mdatasets.load_dataset() before trying to access the Dataset object.\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1179\\u001b[0m     )\\n\',\n',
                '                        "\\u001b[0;31mNotImplementedError\\u001b[0m: Loading a dataset cached in a LocalFileSystem is not supported.",\n',
                "                    ],\n",
                "                },\n",
                "            ],\n",
                '            "source": [\n',
                '                "dataset = load_dataset(\\n",\n',
                "                '    \"json\",\\n',\n",
                "                '    data_files=\"../DATA/processed_dataset_2024-Jan-30_23-29-10.json\",\\n',\n",
                "                '    field=\"data\",\\n',\n",
                '                ")",\n',
                "            ],\n",
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Fine-Tuning\\n"]},\n',
                '        {"cell_type": "markdown", "metadata": {}, "source": []},\n',
                "    ],\n",
                '    "metadata": {\n',
                '        "kernelspec": {\n',
                '            "display_name": "robot-whisperer-lfIYXQcw-py3.10",\n',
                '            "language": "python",\n',
                '            "name": "python3",\n',
                "        },\n",
                '        "language_info": {\n',
                '            "codemirror_mode": {"name": "ipython", "version": 3},\n',
                '            "file_extension": ".py",\n',
                '            "mimetype": "text/x-python",\n',
                '            "name": "python",\n',
                '            "nbconvert_exporter": "python",\n',
                '            "pygments_lexer": "ipython3",\n',
                '            "version": "3.10.13",\n',
                "        },\n",
                "    },\n",
                '    "nbformat": 4,\n',
                '    "nbformat_minor": 2,\n',
                "}",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["# Load model & Tokenizer\n"],
        },
        {
            "cell_type": "code",
            "execution_count": 5,
            "metadata": {},
            "outputs": [
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "9247f78f9c3949b1949207c721e3c698",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "bb47838913be4050892e98a3dff5e963",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "b1691fce8bf04f1eb054a8c82f173855",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "Downloading shards:   0%|          | 0/3 [00:00<?, ?it/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "c223b4be9917420ba66600995e0b9f67",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "11a5e85988c84a7783ce746323bb83b4",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "e09398773fc146ea8ecede8f329bd954",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "8a929f80d6394bc3a1090fbfa36484b7",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "9d21a25ab6e64231ac4593e3078eefed",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "223aed03a73a4ddc85ff82f099475564",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "f1041d3c6e86420bb9a017620b8d34ec",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "659f749ddc7345ddb7ab3968f34965d8",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "fc4e61e6baad4335b1c69dc0998837cf",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
            ],
            "source": [
                'MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"\n',
                'weights_dir = "./weights"\n',
                "\n",
                "bnb_config = BitsAndBytesConfig(\n",
                "    load_in_4bit=True,\n",
                "    bnb_4bit_use_double_quant=True,\n",
                '    bnb_4bit_quant_type="nf4",\n',
                "    bnb_4bit_compute_dtype=torch.bfloat16,\n",
                ")\n",
                "\n",
                "model = AutoModelForCausalLM.from_pretrained(\n",
                "    MODEL_NAME,\n",
                "    device_map=device,\n",
                "    trust_remote_code=True,\n",
                "    quantization_config=bnb_config,\n",
                "    cache_dir=weights_dir,\n",
                ")\n",
                "\n",
                "tokenizer = AutoTokenizer.from_pretrained(\n",
                "    MODEL_NAME, device=device, cache_dir=weights_dir\n",
                ")\n",
                "tokenizer.pad_token = tokenizer.eos_token",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 9,
            "metadata": {},
            "outputs": [],
            "source": [
                "def print_trainable_parameters(model) -> None:\n",
                '    """Prints the number of trainable parameters in the model."""\n',
                "    trainable_params = 0\n",
                "    all_param = 0\n",
                "    for _, param in model.named_parameters():\n",
                "        all_param += param.numel()\n",
                "        if param.requires_grad:\n",
                "            trainable_params += param.numel()\n",
                "\n",
                "    print(\n",
                '        f"Trainable params num: {trainable_params:,} perzentage: {100 * trainable_params / all_param} || All params: {all_param:,}"\n',
                "    )",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 10,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Trainable params num: 262,410,240 perzentage: 6.993743675173274 || All params: 3,752,071,168\n"
                    ],
                }
            ],
            "source": ["print_trainable_parameters(model)"],
        },
        {
            "cell_type": "code",
            "execution_count": 11,
            "metadata": {},
            "outputs": [],
            "source": [
                "model.gradient_checkpointing_enable()\n",
                "model = prepare_model_for_kbit_training(model)",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 13,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Trainable params num: 42,520,576 perzentage: 1.1205573318192514 || All params: 3,794,591,744\n"
                    ],
                }
            ],
            "source": [
                "config = LoraConfig(\n",
                "    r=16,\n",
                "    lora_alpha=32,\n",
                "    target_modules=[\n",
                '        "q_proj",\n',
                '        "k_proj",\n',
                '        "v_proj",\n',
                '        "o_proj",\n',
                '        "gate_proj",\n',
                '        "up_proj",\n',
                '        "down_proj",\n',
                '        "lm_head",\n',
                "    ],\n",
                "    lora_dropout=0.05,\n",
                '    bias="none",\n',
                '    task_type="CAUSAL_LM",\n',
                ")\n",
                "\n",
                "model = get_peft_model(model, config)\n",
                "print_trainable_parameters(model)",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["# Test original model\n"],
        },
        {
            "cell_type": "code",
            "execution_count": 34,
            "metadata": {},
            "outputs": [],
            "source": [
                'prompt_template_large = """\n',
                "<s>[INST]\n",
                "# TASK DESCRIPTION\n",
                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\n",
                "\n",
                "# JSON PARAMETER VALUES\n",
                '- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\n',
                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\n",
                "- input_name: Name of the input parameter for the function.\n",
                "- input_value: Value for the function input parameter.\n",
                "- inputs: Array of input_name, input_value pairs, completing the function call.\n",
                "\n",
                "# FUNCTION MEANING\n",
                "- move_tcp: Moves robot's tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\n",
                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\n",
                "- get_joint_values: Retrieves current robot joint status. No input parameters.\n",
                "\n",
                "# RESPONSE FORMAT\n",
                "- Include only necessary functions from the query.\n",
                "- Maintain the order of functions as in the query.\n",
                "- Note any uncertainties in case of ambiguous queries.\n",
                "\n",
                "# ADDITIONAL GUIDANCE\n",
                "- Focus on verbs and technical terms to determine the function.\n",
                "- Indicate if a query falls outside the available functions.\n",
                "- Consider practical aspects of robotic operations.\n",
                '- Treat "base" as the first robot "joint".\n',
                "\n",
                "# JSON FORMAT\n",
                "{{\n",
                '    "functions": [\n',
                "        {{\n",
                '            "function_name": "<name>",\n',
                '            "inputs": [\n',
                '                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\n',
                '                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\n',
                "            ]\n",
                "        }},\n",
                "        {{\n",
                '            "function_name": "<name>",\n',
                '            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\n',
                "        }}\n",
                "    ]\n",
                "}}\n",
                "\n",
                "USER QUERY: {user_query}\n",
                "[/INST]\n",
                '""".strip()',
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 51,
            "metadata": {},
            "outputs": [],
            "source": [
                'prompt_template_small = """\n',
                "<s>[INST]\n",
                "# TASK DESCRIPTION\n",
                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\n",
                "\n",
                "# JSON PARAMETER VALUES\n",
                '- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\n',
                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\n",
                "- input_name: Name of the input parameter for the function.\n",
                "- input_value: Value for the function input parameter.\n",
                "- inputs: Array of input_name, input_value pairs, completing the function call.\n",
                "\n",
                "# FUNCTION MEANING\n",
                "- move_tcp: Moves robot's tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\n",
                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\n",
                "- get_joint_values: Retrieves current robot joint status. No input parameters.\n",
                "\n",
                "# JSON FORMAT\n",
                "{{\n",
                '    "functions": [\n',
                "        {{\n",
                '            "function_name": "<name>",\n',
                '            "inputs": [\n',
                '                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\n',
                '                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\n',
                "            ]\n",
                "        }},\n",
                "        {{\n",
                '            "function_name": "<name>",\n',
                '            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\n',
                "        }}\n",
                "    ]\n",
                "}}\n",
                "\n",
                "USER QUERY: {user_query}\n",
                "[/INST]\n",
                '""".strip()',
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 75,
            "metadata": {},
            "outputs": [],
            "source": [
                "{\n",
                '    "cells": [\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 2,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "import os\\n",\n',
                '                "import json\\n",\n',
                '                "from pprint import pprint\\n",\n',
                '                "\\n",\n',
                '                "import torch\\n",\n',
                '                "import torch.nn as nn\\n",\n',
                '                "\\n",\n',
                '                "from datasets import load_dataset, DatasetBuilder\\n",\n',
                '                "\\n",\n',
                '                "import transformers\\n",\n',
                '                "from transformers import (\\n",\n',
                '                "    AutoConfig,\\n",\n',
                '                "    AutoModelForCausalLM,\\n",\n',
                '                "    AutoTokenizer,\\n",\n',
                '                "    BitsAndBytesConfig,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "from peft import (\\n",\n',
                '                "    LoraConfig,\\n",\n',
                '                "    PeftConfig,\\n",\n',
                '                "    PeftModel,\\n",\n',
                '                "    get_peft_model,\\n",\n',
                '                "    prepare_model_for_kbit_training,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "from huggingface_hub import notebook_login",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 2,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "12826930970c474a8c7b0feac6339b90",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "VBox(children=(HTML(value=\'<center> <img\\\\nsrc=https://huggingface.co/front/assets/huggingface_logo-noborder.sv…"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                }\n",
                "            ],\n",
                '            "source": ["notebook_login()"],\n',
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Set running device\\n"]},\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 3,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": ["[INFO] Currently running device: cuda\\n"],\n',
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                \'device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\\n\',\n',
                '                \'p   rint(f"[INFO] Currently running device: {device}")                                                               "import os\\\\n",\\n\',\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                        \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                              ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                              ,\\n",\n',
                "                \"                                          '',\\n\",\n",
                "                \"                '',\\n\",\n",
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                              \'\\"\\\\n\',\\n",\n',
                '                "                \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                                \'\\",\\\\n\',\\n",\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "                                                ,\\n",\n',
                '                "         ,\\n",\n',
                '                \'                                                                                 \\\'    """""\\\\n\\\',\\n\',\n',
                "                '                                                                                    \"\\\\n\",\\n',\n",
                '                "                        \'\\"\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                              ,\\n",\n',
                '                "                                                                                 ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                                                                                          \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                                    \'\\"none\\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                "                '                        \"\\\\n\",\\n',\n",
                '                "                        ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                                            \'\\"base\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                                                             ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                         \'\\\\\\"\',\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                "                '                                                                                                      \"\\\\n\",\\n',\n",
                '                "                                    \'\\"and\\", \',\\n",\n',
                "                '                                                                                          \"\\\\n\",\\n',\n",
                "                '                                                                                          \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                                        \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                        \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\',\\n",\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                "                '                        \\'\"name\": \",\\\\n\\',\\n',\n",
                '                "                                    ,\\n",\n',
                '                "               ,\\n",\n',
                '                "                                                      \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                        \'\\"pt\\"\\\\n\',\\n",\n',
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                "                \"                                                                                                                                          '',\\n\",\n",
                "                \"                                    '',\\n\",\n",
                '                "                  ,\\n",\n',
                '                "               ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                                          \'    \\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                '                "                \'\\"data\\",\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                  ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "markdown",\n',
                '            "metadata": {},\n',
                '            "source": ["# Load model & Tokenizer\\n"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 5,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "9247f78f9c3949b1949207c721e3c698",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "bb47838913be4050892e98a3dff5e963",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "b1691fce8bf04f1eb054a8c82f173855",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Downloading shards:   0%|          | 0/3 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "c223b4be9917420ba66600995e0b9f67",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "11a5e85988c84a7783ce746323bb83b4",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "e09398773fc146ea8ecede8f329bd954",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "8a929f80d6394bc3a1090fbfa36484b7",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "9d21a25ab6e64231ac4593e3078eefed",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "223aed03a73a4ddc85ff82f099475564",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "f1041d3c6e86420bb9a017620b8d34ec",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "659f749ddc7345ddb7ab3968f34965d8",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "fc4e61e6baad4335b1c69dc0998837cf",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "            ],\n",
                '            "source": [\n',
                "                'MODEL_NAME = \"mistralai/Mistral-7B-Instruct-v0.2\"\\n',\n",
                "                'weights_dir = \"./weights\"\\n',\n",
                '                "\\n",\n',
                '                "bnb_config = BitsAndBytesConfig(\\n",\n',
                '                "    load_in_4bit=True,\\n",\n',
                '                "    bnb_4bit_use_double_quant=True,\\n",\n',
                "                '    bnb_4bit_quant_type=\"nf4\",\\n',\n",
                '                "    bnb_4bit_compute_dtype=torch.bfloat16,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "model = AutoModelForCausalLM.from_pretrained(\\n",\n',
                '                "    MODEL_NAME,\\n",\n',
                '                "    device_map=device,\\n",\n',
                '                "    trust_remote_code=True,\\n",\n',
                '                "    quantization_config=bnb_config,\\n",\n',
                '                "    cache_dir=weights_dir,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "tokenizer = AutoTokenizer.from_pretrained(\\n",\n',
                '                "    MODEL_NAME, device=device, cache_dir=weights_dir\\n",\n',
                '                ")\\n",\n',
                '                "tokenizer.pad_token = tokenizer.eos_token",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 9,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "def print_trainable_parameters(model) -> None:\\n",\n',
                '                \'    """Prints the number of trainable parameters in the model."""\\n\',\n',
                '                "    trainable_params = 0\\n",\n',
                '                "    all_param = 0\\n",\n',
                '                "    for _, param in model.named_parameters():\\n",\n',
                '                "        all_param += param.numel()\\n",\n',
                '                "        if param.requires_grad:\\n",\n',
                '                "            trainable_params += param.numel()\\n",\n',
                '                "\\n",\n',
                '                "    print(\\n",\n',
                "                '        f\"Trainable params num: {trainable_params:,} perzentage: {100 * trainable_params / all_param} || All params: {all_param:,}\"\\n',\n",
                '                "    )",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 10,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "Trainable params num: 262,410,240 perzentage: 6.993743675173274 || All params: 3,752,071,168\\n"\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": ["print_trainable_parameters(model)"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 11,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "model.gradient_checkpointing_enable()\\n",\n',
                '                "model = prepare_model_for_kbit_training(model)",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 13,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "Trainable params num: 42,520,576 perzentage: 1.1205573318192514 || All params: 3,794,591,744\\n"\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                "config = LoraConfig(\\n",\n',
                '                "    r=16,\\n",\n',
                '                "    lora_alpha=32,\\n",\n',
                '                "    target_modules=[\\n",\n',
                "                '        \"q_proj\",\\n',\n",
                "                '        \"k_proj\",\\n',\n",
                "                '        \"v_proj\",\\n',\n",
                "                '        \"o_proj\",\\n',\n",
                "                '        \"gate_proj\",\\n',\n",
                "                '        \"up_proj\",\\n',\n",
                "                '        \"down_proj\",\\n',\n",
                "                '        \"lm_head\",\\n',\n",
                '                "    ],\\n",\n',
                '                "    lora_dropout=0.05,\\n",\n',
                "                '    bias=\"none\",\\n',\n",
                "                '    task_type=\"CAUSAL_LM\",\\n',\n",
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "model = get_peft_model(model, config)\\n",\n',
                '                "print_trainable_parameters(model)",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "markdown",\n',
                '            "metadata": {},\n',
                '            "source": ["# Test original model\\n"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 34,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                \'prompt_template_large = """\\n\',\n',
                '                "<s>[INST]\\n",\n',
                '                "# TASK DESCRIPTION\\n",\n',
                '                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                "\\n",\n',
                '                "# JSON PARAMETER VALUES\\n",\n',
                '                \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                "- input_name: Name of the input parameter for the function.\\n",\n',
                '                "- input_value: Value for the function input parameter.\\n",\n',
                '                "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                "\\n",\n',
                '                "# FUNCTION MEANING\\n",\n',
                '                "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                "\\n",\n',
                '                "# RESPONSE FORMAT\\n",\n',
                '                "- Include only necessary functions from the query.\\n",\n',
                '                "- Maintain the order of functions as in the query.\\n",\n',
                '                "- Note any uncertainties in case of ambiguous queries.\\n",\n',
                '                "\\n",\n',
                '                "# ADDITIONAL GUIDANCE\\n",\n',
                '                "- Focus on verbs and technical terms to determine the function.\\n",\n',
                '                "- Indicate if a query falls outside the available functions.\\n",\n',
                '                "- Consider practical aspects of robotic operations.\\n",\n',
                '                \'- Treat "base" as the first robot "joint".\\n\',\n',
                '                "\\n",\n',
                '                "# JSON FORMAT\\n",\n',
                '                "{{\\n",\n',
                "                '    \"functions\": [\\n',\n",
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                "                '            \"inputs\": [\\n',\n",
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\\n\',\n',
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\\n\',\n',
                '                "            ]\\n",\n',
                '                "        }},\\n",\n',
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                '                \'            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\\n\',\n',
                '                "        }}\\n",\n',
                '                "    ]\\n",\n',
                '                "}}\\n",\n',
                '                "\\n",\n',
                '                "USER QUERY: {user_query}\\n",\n',
                '                "[/INST]\\n",\n',
                '                \'""".strip()\',\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 51,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                \'prompt_template_small = """\\n\',\n',
                '                "<s>[INST]\\n",\n',
                '                "# TASK DESCRIPTION\\n",\n',
                '                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                "\\n",\n',
                '                "# JSON PARAMETER VALUES\\n",\n',
                '                \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                "- input_name: Name of the input parameter for the function.\\n",\n',
                '                "- input_value: Value for the function input parameter.\\n",\n',
                '                "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                "\\n",\n',
                '                "# FUNCTION MEANING\\n",\n',
                '                "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                "\\n",\n',
                '                "# JSON FORMAT\\n",\n',
                '                "{{\\n",\n',
                "                '    \"functions\": [\\n',\n",
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                "                '            \"inputs\": [\\n',\n",
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\\n\',\n',
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\\n\',\n',
                '                "            ]\\n",\n',
                '                "        }},\\n",\n',
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                '                \'            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\\n\',\n',
                '                "        }}\\n",\n',
                '                "    ]\\n",\n',
                '                "}}\\n",\n',
                '                "\\n",\n',
                '                "USER QUERY: {user_query}\\n",\n',
                '                "[/INST]\\n",\n',
                '                \'""".strip()\',\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 75,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "generation_config = model.generation_config\\n",\n',
                '                "g   eneration_config.max_new_tokens = 200\\n",\n',
                '                "ge      neration_config.temperature = 0.7\\n",\n',
                '                "gen         eration_config.top_p = 0.7\\n",\n',
                '                "gen         eration_config.num_return_sequences = 1\\n",\n',
                '                "         generation_config.pad_token_id = tokenizer.eos_token_id\\n",\n',
                "                'gen         eration_config.eos_token_id = tokenizer.eos_token_id                     \"import os\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                        \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                              ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                              ,\\n",\n',
                "                \"                                          '',\\n\",\n",
                "                \"                '',\\n\",\n",
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                              \'\\"\\\\n\',\\n",\n',
                '                "                \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                                \'\\",\\\\n\',\\n",\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "                                                ,\\n",\n',
                '                "         ,\\n",\n',
                '                \'                                                                                 \\\'    """""\\\\n\\\',\\n\',\n',
                "                '                                                                                    \"\\\\n\",\\n',\n",
                '                "                        \'\\"\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                              ,\\n",\n',
                '                "                                                                                 ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                                                                                          \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                                    \'\\"none\\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                "                '                        \"\\\\n\",\\n',\n",
                '                "                        ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                                            \'\\"base\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                                                             ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                         \'\\\\\\"\',\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                "                '                                                                                                      \"\\\\n\",\\n',\n",
                '                "                                    \'\\"and\\", \',\\n",\n',
                "                '                                                                                          \"\\\\n\",\\n',\n",
                "                '                                                                                          \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                                        \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                        \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\',\\n",\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                "                '                        \\'\"name\": \",\\\\n\\',\\n',\n",
                '                "                                    ,\\n",\n',
                '                "               ,\\n",\n',
                '                "                                                      \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                        \'\\"pt\\"\\\\n\',\\n",\n',
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                "                \"                                                                                                                                          '',\\n\",\n",
                "                \"                                    '',\\n\",\n",
                '                "                  ,\\n",\n',
                '                "               ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                                          \'    \\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                '                "                \'\\"data\\",\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                  ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 76,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\'user_query = "Obtain the status of the robot\\\'s third joint"\'],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 77,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "[INST]\\n",\n',
                '                        "# TASK DESCRIPTION\\n",\n',
                '                        "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                        "\\n",\n',
                '                        "# JSON PARAMETER VALUES\\n",\n',
                '                        \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                        "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                        "- input_name: Name of the input parameter for the function.\\n",\n',
                '                        "- input_value: Value for the function input parameter.\\n",\n',
                '                        "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                        "\\n",\n',
                '                        "# FUNCTION MEANING\\n",\n',
                '                        "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                        "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                        "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                        "\\n",\n',
                '                        "# JSON FORMAT\\n",\n',
                '                        "{\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "<name>",\\n\',\n',
                "                        '            \"inputs\": [\\n',\n",
                '                        \'                {"name": "<name>", "value": "<value>", "unit": "<unit>"},\\n\',\n',
                '                        \'                {"name": "<name>", "value": "<value>", "unit": "<unit>"}\\n\',\n',
                '                        "            ]\\n",\n',
                '                        "        },\\n",\n',
                '                        "        {\\n",\n',
                '                        \'            "function_name": "<name>",\\n\',\n',
                '                        \'            "inputs": [{"name": "<name>", "value": "<value>", "unit": "<unit>"}]\\n\',\n',
                '                        "        }\\n",\n',
                '                        "    ]\\n",\n',
                '                        "}\\n",\n',
                '                        "\\n",\n',
                '                        "USER QUERY: Obtain the status of the robot\'s third joint\\n",\n',
                '                        "[/INST] {\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "get_joint_values",\\n\',\n',
                "                        '            \"inputs\": []\\n',\n",
                '                        "        }\\n",\n',
                '                        "    ]\\n",\n',
                '                        "}\\n",\n',
                '                        "\\n",\n',
                '                        "USER QUERY: Move the robot\'s end effector to position (200, 300, 400) and orientation (0.1, 0.2, 0.3, 0.4)\\n",\n',
                '                        "{\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "move_tcp",\\n\',\n',
                "                        '            \"inputs\": [\\n',\n",
                '                        \'                {"name": "x", "value": 200, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "y", "value": 300, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "z", "value": 400, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "q1",\\n\',\n',
                '                        "CPU times: user 8.04 s, sys: 0 ns, total: 8.04 s\\n",\n',
                '                        "Wall time: 8.04 s\\n",\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                "%%time\\n",\n',
                "                'device = \"cuda:0\"\\n',\n",
                '                "\\n",\n',
                '                "encoding = tokenizer(\\n",\n',
                "                '    prompt_template_small.format(user_query=user_query), return_tensors=\"pt\"\\n',\n",
                '                ").to(device)\\n",\n',
                '                "with torch.inference_mode():\\n",\n',
                '                "    outputs = model.generate(\\n",\n',
                '                "        input_ids=encoding.input_ids,\\n",\n',
                '                "        attention_mask=encoding.attention_mask,\\n",\n',
                '                "        generation_config=generation_config,\\n",\n',
                '                "    )\\n",\n',
                '                "\\n",\n',
                '                "print(tokenizer.decode(outputs[0], skip_special_tokens=True))",\n',
                "            ],\n",
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Prepare data\\n"]},\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 5,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "799088593edf4f5ca4a9d96197c77670",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Downloading data files:   0%|          | 0/1 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "1624861f54584c94a26e6959e1efd8ff",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "7f613b42d76c4638a525ff4ec75f91ec",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Generating train split: 0 examples [00:00, ? examples/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "ename": "NotImplementedError",\n',
                '                    "evalue": "Loading a dataset cached in a LocalFileSystem is not supported.",\n',
                '                    "output_type": "error",\n',
                '                    "traceback": [\n',
                '                        "\\u001b[0;31m---------------------------------------------------------------------------\\u001b[0m",\n',
                '                        "\\u001b[0;31mNotImplementedError\\u001b[0m                       Traceback (most recent call last)",\n',
                '                        \'Cell \\u001b[0;32mIn[5], line 1\\u001b[0m\\n\\u001b[0;32m----> 1\\u001b[0m dataset \\u001b[38;5;241m=\\u001b[39m \\u001b[43mload_dataset\\u001b[49m\\u001b[43m(\\u001b[49m\\n\\u001b[1;32m      2\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43mjson\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      3\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[43mdata_files\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43m../DATA/processed_dataset_2024-Jan-30_23-29-10.json\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      4\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[43mfield\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43mdata\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      5\\u001b[0m \\u001b[43m)\\u001b[49m\\n\',\n',
                '                        "File \\u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/load.py:2149\\u001b[0m, in \\u001b[0;36mload_dataset\\u001b[0;34m(path, name, data_dir, data_files, split, cache_dir, features, download_config, download_mode, verification_mode, ignore_verifications, keep_in_memory, save_infos, revision, token, use_auth_token, task, streaming, num_proc, storage_options, **config_kwargs)\\u001b[0m\\n\\u001b[1;32m   2145\\u001b[0m \\u001b[38;5;66;03m# Build dataset for splits\\u001b[39;00m\\n\\u001b[1;32m   2146\\u001b[0m keep_in_memory \\u001b[38;5;241m=\\u001b[39m (\\n\\u001b[1;32m   2147\\u001b[0m     keep_in_memory \\u001b[38;5;28;01mif\\u001b[39;00m keep_in_memory \\u001b[38;5;129;01mis\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m \\u001b[38;5;28;01mNone\\u001b[39;00m \\u001b[38;5;28;01melse\\u001b[39;00m is_small_dataset(builder_instance\\u001b[38;5;241m.\\u001b[39minfo\\u001b[38;5;241m.\\u001b[39mdataset_size)\\n\\u001b[1;32m   2148\\u001b[0m )\\n\\u001b[0;32m-> 2149\\u001b[0m ds \\u001b[38;5;241m=\\u001b[39m \\u001b[43mbuilder_instance\\u001b[49m\\u001b[38;5;241;43m.\\u001b[39;49m\\u001b[43mas_dataset\\u001b[49m\\u001b[43m(\\u001b[49m\\u001b[43msplit\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43msplit\\u001b[49m\\u001b[43m,\\u001b[49m\\u001b[43m \\u001b[49m\\u001b[43mverification_mode\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43mverification_mode\\u001b[49m\\u001b[43m,\\u001b[49m\\u001b[43m \\u001b[49m\\u001b[43min_memory\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43mkeep_in_memory\\u001b[49m\\u001b[43m)\\u001b[49m\\n\\u001b[1;32m   2150\\u001b[0m \\u001b[38;5;66;03m# Rename and cast features to match task schema\\u001b[39;00m\\n\\u001b[1;32m   2151\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m task \\u001b[38;5;129;01mis\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m \\u001b[38;5;28;01mNone\\u001b[39;00m:\\n\\u001b[1;32m   2152\\u001b[0m     \\u001b[38;5;66;03m# To avoid issuing the same warning twice\\u001b[39;00m\\n",\n',
                '                        \'File \\u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/builder.py:1173\\u001b[0m, in \\u001b[0;36mDatasetBuilder.as_dataset\\u001b[0;34m(self, split, run_post_process, verification_mode, ignore_verifications, in_memory)\\u001b[0m\\n\\u001b[1;32m   1171\\u001b[0m is_local \\u001b[38;5;241m=\\u001b[39m \\u001b[38;5;129;01mnot\\u001b[39;00m is_remote_filesystem(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_fs)\\n\\u001b[1;32m   1172\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m is_local:\\n\\u001b[0;32m-> 1173\\u001b[0m     \\u001b[38;5;28;01mraise\\u001b[39;00m \\u001b[38;5;167;01mNotImplementedError\\u001b[39;00m(\\u001b[38;5;124mf\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mLoading a dataset cached in a \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mtype\\u001b[39m(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_fs)\\u001b[38;5;241m.\\u001b[39m\\u001b[38;5;18m__name__\\u001b[39m\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m is not supported.\\u001b[39m\\u001b[38;5;124m"\\u001b[39m)\\n\\u001b[1;32m   1174\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m os\\u001b[38;5;241m.\\u001b[39mpath\\u001b[38;5;241m.\\u001b[39mexists(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_output_dir):\\n\\u001b[1;32m   1175\\u001b[0m     \\u001b[38;5;28;01mraise\\u001b[39;00m \\u001b[38;5;167;01mFileNotFoundError\\u001b[39;00m(\\n\\u001b[1;32m   1176\\u001b[0m         \\u001b[38;5;124mf\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mDataset \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39mdataset_name\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m: could not find data in \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_output_dir\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m. Please make sure to call \\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1177\\u001b[0m         \\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mbuilder.download_and_prepare(), or use \\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1178\\u001b[0m         \\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mdatasets.load_dataset() before trying to access the Dataset object.\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1179\\u001b[0m     )\\n\',\n',
                '                        "\\u001b[0;31mNotImplementedError\\u001b[0m: Loading a dataset cached in a LocalFileSystem is not supported.",\n',
                "                    ],\n",
                "                },\n",
                "            ],\n",
                '            "source": [\n',
                '                "dataset = load_dataset(\\n",\n',
                "                '    \"json\",\\n',\n",
                "                '    data_files=\"../DATA/processed_dataset_2024-Jan-30_23-29-10.json\",\\n',\n",
                "                '    field=\"data\",\\n',\n",
                '                ")",\n',
                "            ],\n",
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Fine-Tuning\\n"]},\n',
                '        {"cell_type": "markdown", "metadata": {}, "source": []},\n',
                "    ],\n",
                '    "metadata": {\n',
                '        "kernelspec": {\n',
                '            "display_name": "robot-whisperer-lfIYXQcw-py3.10",\n',
                '            "language": "python",\n',
                '            "name": "python3",\n',
                "        },\n",
                '        "language_info": {\n',
                '            "codemirror_mode": {"name": "ipython", "version": 3},\n',
                '            "file_extension": ".py",\n',
                '            "mimetype": "text/x-python",\n',
                '            "name": "python",\n',
                '            "nbconvert_exporter": "python",\n',
                '            "pygments_lexer": "ipython3",\n',
                '            "version": "3.10.13",\n',
                "        },\n",
                "    },\n",
                '    "nbformat": 4,\n',
                '    "nbformat_minor": 2,\n',
                "}",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 76,
            "metadata": {},
            "outputs": [],
            "source": ['user_query = "Obtain the status of the robot\'s third joint"'],
        },
        {
            "cell_type": "code",
            "execution_count": 77,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "[INST]\n",
                        "# TASK DESCRIPTION\n",
                        "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\n",
                        "\n",
                        "# JSON PARAMETER VALUES\n",
                        '- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\n',
                        "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\n",
                        "- input_name: Name of the input parameter for the function.\n",
                        "- input_value: Value for the function input parameter.\n",
                        "- inputs: Array of input_name, input_value pairs, completing the function call.\n",
                        "\n",
                        "# FUNCTION MEANING\n",
                        "- move_tcp: Moves robot's tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\n",
                        "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\n",
                        "- get_joint_values: Retrieves current robot joint status. No input parameters.\n",
                        "\n",
                        "# JSON FORMAT\n",
                        "{\n",
                        '    "functions": [\n',
                        "        {\n",
                        '            "function_name": "<name>",\n',
                        '            "inputs": [\n',
                        '                {"name": "<name>", "value": "<value>", "unit": "<unit>"},\n',
                        '                {"name": "<name>", "value": "<value>", "unit": "<unit>"}\n',
                        "            ]\n",
                        "        },\n",
                        "        {\n",
                        '            "function_name": "<name>",\n',
                        '            "inputs": [{"name": "<name>", "value": "<value>", "unit": "<unit>"}]\n',
                        "        }\n",
                        "    ]\n",
                        "}\n",
                        "\n",
                        "USER QUERY: Obtain the status of the robot's third joint\n",
                        "[/INST] {\n",
                        '    "functions": [\n',
                        "        {\n",
                        '            "function_name": "get_joint_values",\n',
                        '            "inputs": []\n',
                        "        }\n",
                        "    ]\n",
                        "}\n",
                        "\n",
                        "USER QUERY: Move the robot's end effector to position (200, 300, 400) and orientation (0.1, 0.2, 0.3, 0.4)\n",
                        "{\n",
                        '    "functions": [\n',
                        "        {\n",
                        '            "function_name": "move_tcp",\n',
                        '            "inputs": [\n',
                        '                {"name": "x", "value": 200, "unit": "mm"},\n',
                        '                {"name": "y", "value": 300, "unit": "mm"},\n',
                        '                {"name": "z", "value": 400, "unit": "mm"},\n',
                        '                {"name": "q1",\n',
                        "CPU times: user 8.04 s, sys: 0 ns, total: 8.04 s\n",
                        "Wall time: 8.04 s\n",
                    ],
                }
            ],
            "source": [
                "%%time\n",
                'device = "cuda:0"\n',
                "\n",
                "encoding = tokenizer(\n",
                '    prompt_template_small.format(user_query=user_query), return_tensors="pt"\n',
                ").to(device)\n",
                "with torch.inference_mode():\n",
                "    outputs = model.generate(\n",
                "        input_ids=encoding.input_ids,\n",
                "        attention_mask=encoding.attention_mask,\n",
                "        generation_config=generation_config,\n",
                "    )\n",
                "\n",
                "print(tokenizer.decode(outputs[0], skip_special_tokens=True))",
            ],
        },
        {"cell_type": "markdown", "metadata": {}, "source": ["# Prepare data\n"]},
        {
            "cell_type": "code",
            "execution_count": 5,
            "metadata": {},
            "outputs": [
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "799088593edf4f5ca4a9d96197c77670",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "Downloading data files:   0%|          | 0/1 [00:00<?, ?it/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "1624861f54584c94a26e6959e1efd8ff",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "7f613b42d76c4638a525ff4ec75f91ec",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "Generating train split: 0 examples [00:00, ? examples/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "ename": "NotImplementedError",
                    "evalue": "Loading a dataset cached in a LocalFileSystem is not supported.",
                    "output_type": "error",
                    "traceback": [
                        "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
                        "\u001b[0;31mNotImplementedError\u001b[0m                       Traceback (most recent call last)",
                        'Cell \u001b[0;32mIn[5], line 1\u001b[0m\n\u001b[0;32m----> 1\u001b[0m dataset \u001b[38;5;241m=\u001b[39m \u001b[43mload_dataset\u001b[49m\u001b[43m(\u001b[49m\n\u001b[1;32m      2\u001b[0m \u001b[43m    \u001b[49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[38;5;124;43mjson\u001b[39;49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[43m,\u001b[49m\n\u001b[1;32m      3\u001b[0m \u001b[43m    \u001b[49m\u001b[43mdata_files\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[38;5;124;43m../DATA/processed_dataset_2024-Jan-30_23-29-10.json\u001b[39;49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[43m,\u001b[49m\n\u001b[1;32m      4\u001b[0m \u001b[43m    \u001b[49m\u001b[43mfield\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[38;5;124;43mdata\u001b[39;49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[43m,\u001b[49m\n\u001b[1;32m      5\u001b[0m \u001b[43m)\u001b[49m\n',
                        "File \u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/load.py:2149\u001b[0m, in \u001b[0;36mload_dataset\u001b[0;34m(path, name, data_dir, data_files, split, cache_dir, features, download_config, download_mode, verification_mode, ignore_verifications, keep_in_memory, save_infos, revision, token, use_auth_token, task, streaming, num_proc, storage_options, **config_kwargs)\u001b[0m\n\u001b[1;32m   2145\u001b[0m \u001b[38;5;66;03m# Build dataset for splits\u001b[39;00m\n\u001b[1;32m   2146\u001b[0m keep_in_memory \u001b[38;5;241m=\u001b[39m (\n\u001b[1;32m   2147\u001b[0m     keep_in_memory \u001b[38;5;28;01mif\u001b[39;00m keep_in_memory \u001b[38;5;129;01mis\u001b[39;00m \u001b[38;5;129;01mnot\u001b[39;00m \u001b[38;5;28;01mNone\u001b[39;00m \u001b[38;5;28;01melse\u001b[39;00m is_small_dataset(builder_instance\u001b[38;5;241m.\u001b[39minfo\u001b[38;5;241m.\u001b[39mdataset_size)\n\u001b[1;32m   2148\u001b[0m )\n\u001b[0;32m-> 2149\u001b[0m ds \u001b[38;5;241m=\u001b[39m \u001b[43mbuilder_instance\u001b[49m\u001b[38;5;241;43m.\u001b[39;49m\u001b[43mas_dataset\u001b[49m\u001b[43m(\u001b[49m\u001b[43msplit\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43msplit\u001b[49m\u001b[43m,\u001b[49m\u001b[43m \u001b[49m\u001b[43mverification_mode\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mverification_mode\u001b[49m\u001b[43m,\u001b[49m\u001b[43m \u001b[49m\u001b[43min_memory\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mkeep_in_memory\u001b[49m\u001b[43m)\u001b[49m\n\u001b[1;32m   2150\u001b[0m \u001b[38;5;66;03m# Rename and cast features to match task schema\u001b[39;00m\n\u001b[1;32m   2151\u001b[0m \u001b[38;5;28;01mif\u001b[39;00m task \u001b[38;5;129;01mis\u001b[39;00m \u001b[38;5;129;01mnot\u001b[39;00m \u001b[38;5;28;01mNone\u001b[39;00m:\n\u001b[1;32m   2152\u001b[0m     \u001b[38;5;66;03m# To avoid issuing the same warning twice\u001b[39;00m\n",
                        'File \u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/builder.py:1173\u001b[0m, in \u001b[0;36mDatasetBuilder.as_dataset\u001b[0;34m(self, split, run_post_process, verification_mode, ignore_verifications, in_memory)\u001b[0m\n\u001b[1;32m   1171\u001b[0m is_local \u001b[38;5;241m=\u001b[39m \u001b[38;5;129;01mnot\u001b[39;00m is_remote_filesystem(\u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39m_fs)\n\u001b[1;32m   1172\u001b[0m \u001b[38;5;28;01mif\u001b[39;00m \u001b[38;5;129;01mnot\u001b[39;00m is_local:\n\u001b[0;32m-> 1173\u001b[0m     \u001b[38;5;28;01mraise\u001b[39;00m \u001b[38;5;167;01mNotImplementedError\u001b[39;00m(\u001b[38;5;124mf\u001b[39m\u001b[38;5;124m"\u001b[39m\u001b[38;5;124mLoading a dataset cached in a \u001b[39m\u001b[38;5;132;01m{\u001b[39;00m\u001b[38;5;28mtype\u001b[39m(\u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39m_fs)\u001b[38;5;241m.\u001b[39m\u001b[38;5;18m__name__\u001b[39m\u001b[38;5;132;01m}\u001b[39;00m\u001b[38;5;124m is not supported.\u001b[39m\u001b[38;5;124m"\u001b[39m)\n\u001b[1;32m   1174\u001b[0m \u001b[38;5;28;01mif\u001b[39;00m \u001b[38;5;129;01mnot\u001b[39;00m os\u001b[38;5;241m.\u001b[39mpath\u001b[38;5;241m.\u001b[39mexists(\u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39m_output_dir):\n\u001b[1;32m   1175\u001b[0m     \u001b[38;5;28;01mraise\u001b[39;00m \u001b[38;5;167;01mFileNotFoundError\u001b[39;00m(\n\u001b[1;32m   1176\u001b[0m         \u001b[38;5;124mf\u001b[39m\u001b[38;5;124m"\u001b[39m\u001b[38;5;124mDataset \u001b[39m\u001b[38;5;132;01m{\u001b[39;00m\u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39mdataset_name\u001b[38;5;132;01m}\u001b[39;00m\u001b[38;5;124m: could not find data in \u001b[39m\u001b[38;5;132;01m{\u001b[39;00m\u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39m_output_dir\u001b[38;5;132;01m}\u001b[39;00m\u001b[38;5;124m. Please make sure to call \u001b[39m\u001b[38;5;124m"\u001b[39m\n\u001b[1;32m   1177\u001b[0m         \u001b[38;5;124m"\u001b[39m\u001b[38;5;124mbuilder.download_and_prepare(), or use \u001b[39m\u001b[38;5;124m"\u001b[39m\n\u001b[1;32m   1178\u001b[0m         \u001b[38;5;124m"\u001b[39m\u001b[38;5;124mdatasets.load_dataset() before trying to access the Dataset object.\u001b[39m\u001b[38;5;124m"\u001b[39m\n\u001b[1;32m   1179\u001b[0m     )\n',
                        "\u001b[0;31mNotImplementedError\u001b[0m: Loading a dataset cached in a LocalFileSystem is not supported.",
                    ],
                },
            ],
            "source": [
                "dataset = load_dataset(\n",
                '    "json",\n',
                '    data_files="../DATA/processed_dataset_2024-Jan-30_23-29-10.json",\n',
                '    field="data",\n',
                ")",
            ],
        },
        {"cell_type": "markdown", "metadata": {}, "source": ["# Fine-Tuning\n"]},
        {"cell_type": "markdown", "metadata": {}, "source": []},
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "robot-whisperer-lfIYXQcw-py3.10",
            "language": "python",
            "name": "python3",
        },
        "language_info": {
            "codemirror_mode": {"name": "ipython", "version": 3},
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.10.13",
        },
    },
    "nbformat": 4,
    "nbformat_minor": 2,
}

[INFO] Currently running device: cuda


# Load model & Tokenizer


In [5]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
weights_dir = "./weights"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map=device,
    trust_remote_code=True,
    quantization_config=bnb_config,
    cache_dir=weights_dir,
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, device=device, cache_dir=weights_dir
)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
def print_trainable_parameters(model) -> None:
    """Prints the number of trainable parameters in the model."""
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(
        f"Trainable params num: {trainable_params:,} perzentage: {100 * trainable_params / all_param} || All params: {all_param:,}"
    )

In [10]:
print_trainable_parameters(model)

Trainable params num: 262,410,240 perzentage: 6.993743675173274 || All params: 3,752,071,168


In [11]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [13]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

Trainable params num: 42,520,576 perzentage: 1.1205573318192514 || All params: 3,794,591,744


# Test original model


In [34]:
prompt_template_large = """
<s>[INST]
# TASK DESCRIPTION
Translate user queries about industrial robotic operations into JSON outputs for specific function calls.

# JSON PARAMETER VALUES
- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".
- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.
- input_name: Name of the input parameter for the function.
- input_value: Value for the function input parameter.
- inputs: Array of input_name, input_value pairs, completing the function call.

# FUNCTION MEANING
- move_tcp: Moves robot's tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).
- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).
- get_joint_values: Retrieves current robot joint status. No input parameters.

# RESPONSE FORMAT
- Include only necessary functions from the query.
- Maintain the order of functions as in the query.
- Note any uncertainties in case of ambiguous queries.

# ADDITIONAL GUIDANCE
- Focus on verbs and technical terms to determine the function.
- Indicate if a query falls outside the available functions.
- Consider practical aspects of robotic operations.
- Treat "base" as the first robot "joint".

# JSON FORMAT
{{
    "functions": [
        {{
            "function_name": "<name>",
            "inputs": [
                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},
                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}
            ]
        }},
        {{
            "function_name": "<name>",
            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]
        }}
    ]
}}

USER QUERY: {user_query}
[/INST]
""".strip()

In [51]:
prompt_template_small = """
<s>[INST]
# TASK DESCRIPTION
Translate user queries about industrial robotic operations into JSON outputs for specific function calls.

# JSON PARAMETER VALUES
- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".
- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.
- input_name: Name of the input parameter for the function.
- input_value: Value for the function input parameter.
- inputs: Array of input_name, input_value pairs, completing the function call.

# FUNCTION MEANING
- move_tcp: Moves robot's tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).
- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).
- get_joint_values: Retrieves current robot joint status. No input parameters.

# JSON FORMAT
{{
    "functions": [
        {{
            "function_name": "<name>",
            "inputs": [
                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},
                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}
            ]
        }},
        {{
            "function_name": "<name>",
            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]
        }}
    ]
}}

USER QUERY: {user_query}
[/INST]
""".strip()

In [75]:
{
    "cells": [
        {
            "cell_type": "code",
            "execution_count": 2,
            "metadata": {},
            "outputs": [],
            "source": [
                "import os\n",
                "import json\n",
                "from pprint import pprint\n",
                "\n",
                "import torch\n",
                "import torch.nn as nn\n",
                "\n",
                "from datasets import load_dataset, DatasetBuilder\n",
                "\n",
                "import transformers\n",
                "from transformers import (\n",
                "    AutoConfig,\n",
                "    AutoModelForCausalLM,\n",
                "    AutoTokenizer,\n",
                "    BitsAndBytesConfig,\n",
                ")\n",
                "\n",
                "from peft import (\n",
                "    LoraConfig,\n",
                "    PeftConfig,\n",
                "    PeftModel,\n",
                "    get_peft_model,\n",
                "    prepare_model_for_kbit_training,\n",
                ")\n",
                "\n",
                "from huggingface_hub import notebook_login",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 2,
            "metadata": {},
            "outputs": [
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "12826930970c474a8c7b0feac6339b90",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "VBox(children=(HTML(value='<center> <img\\nsrc=https://huggingface.co/front/assets/huggingface_logo-noborder.sv…"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                }
            ],
            "source": ["notebook_login()"],
        },
        {"cell_type": "markdown", "metadata": {}, "source": ["# Set running device\n"]},
        {
            "cell_type": "code",
            "execution_count": 3,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": ["[INFO] Currently running device: cuda\n"],
                }
            ],
            "source": [
                "{\n",
                '    "cells": [\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 2,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "import os\\n",\n',
                '                "import json\\n",\n',
                '                "from pprint import pprint\\n",\n',
                '                "\\n",\n',
                '                "import torch\\n",\n',
                '                "import torch.nn as nn\\n",\n',
                '                "\\n",\n',
                '                "from datasets import load_dataset, DatasetBuilder\\n",\n',
                '                "\\n",\n',
                '                "import transformers\\n",\n',
                '                "from transformers import (\\n",\n',
                '                "    AutoConfig,\\n",\n',
                '                "    AutoModelForCausalLM,\\n",\n',
                '                "    AutoTokenizer,\\n",\n',
                '                "    BitsAndBytesConfig,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "from peft import (\\n",\n',
                '                "    LoraConfig,\\n",\n',
                '                "    PeftConfig,\\n",\n',
                '                "    PeftModel,\\n",\n',
                '                "    get_peft_model,\\n",\n',
                '                "    prepare_model_for_kbit_training,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "from huggingface_hub import notebook_login",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 2,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "12826930970c474a8c7b0feac6339b90",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "VBox(children=(HTML(value=\'<center> <img\\\\nsrc=https://huggingface.co/front/assets/huggingface_logo-noborder.sv…"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                }\n",
                "            ],\n",
                '            "source": ["notebook_login()"],\n',
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Set running device\\n"]},\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 3,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": ["[INFO] Currently running device: cuda\\n"],\n',
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                \'device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\\n\',\n',
                '                \'p   rint(f"[INFO] Currently running device: {device}")                                                               "import os\\\\n",\\n\',\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                        \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                              ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                              ,\\n",\n',
                "                \"                                          '',\\n\",\n",
                "                \"                '',\\n\",\n",
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                              \'\\"\\\\n\',\\n",\n',
                '                "                \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                                \'\\",\\\\n\',\\n",\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "                                                ,\\n",\n',
                '                "         ,\\n",\n',
                '                \'                                                                                 \\\'    """""\\\\n\\\',\\n\',\n',
                "                '                                                                                    \"\\\\n\",\\n',\n",
                '                "                        \'\\"\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                              ,\\n",\n',
                '                "                                                                                 ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                                                                                          \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                                    \'\\"none\\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                "                '                        \"\\\\n\",\\n',\n",
                '                "                        ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                                            \'\\"base\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                                                             ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                         \'\\\\\\"\',\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                "                '                                                                                                      \"\\\\n\",\\n',\n",
                '                "                                    \'\\"and\\", \',\\n",\n',
                "                '                                                                                          \"\\\\n\",\\n',\n",
                "                '                                                                                          \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                                        \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                        \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\',\\n",\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                "                '                        \\'\"name\": \",\\\\n\\',\\n',\n",
                '                "                                    ,\\n",\n',
                '                "               ,\\n",\n',
                '                "                                                      \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                        \'\\"pt\\"\\\\n\',\\n",\n',
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                "                \"                                                                                                                                          '',\\n\",\n",
                "                \"                                    '',\\n\",\n",
                '                "                  ,\\n",\n',
                '                "               ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                                          \'    \\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                '                "                \'\\"data\\",\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                  ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "markdown",\n',
                '            "metadata": {},\n',
                '            "source": ["# Load model & Tokenizer\\n"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 5,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "9247f78f9c3949b1949207c721e3c698",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "bb47838913be4050892e98a3dff5e963",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "b1691fce8bf04f1eb054a8c82f173855",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Downloading shards:   0%|          | 0/3 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "c223b4be9917420ba66600995e0b9f67",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "11a5e85988c84a7783ce746323bb83b4",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "e09398773fc146ea8ecede8f329bd954",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "8a929f80d6394bc3a1090fbfa36484b7",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "9d21a25ab6e64231ac4593e3078eefed",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "223aed03a73a4ddc85ff82f099475564",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "f1041d3c6e86420bb9a017620b8d34ec",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "659f749ddc7345ddb7ab3968f34965d8",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "fc4e61e6baad4335b1c69dc0998837cf",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "            ],\n",
                '            "source": [\n',
                "                'MODEL_NAME = \"mistralai/Mistral-7B-Instruct-v0.2\"\\n',\n",
                "                'weights_dir = \"./weights\"\\n',\n",
                '                "\\n",\n',
                '                "bnb_config = BitsAndBytesConfig(\\n",\n',
                '                "    load_in_4bit=True,\\n",\n',
                '                "    bnb_4bit_use_double_quant=True,\\n",\n',
                "                '    bnb_4bit_quant_type=\"nf4\",\\n',\n",
                '                "    bnb_4bit_compute_dtype=torch.bfloat16,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "model = AutoModelForCausalLM.from_pretrained(\\n",\n',
                '                "    MODEL_NAME,\\n",\n',
                '                "    device_map=device,\\n",\n',
                '                "    trust_remote_code=True,\\n",\n',
                '                "    quantization_config=bnb_config,\\n",\n',
                '                "    cache_dir=weights_dir,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "tokenizer = AutoTokenizer.from_pretrained(\\n",\n',
                '                "    MODEL_NAME, device=device, cache_dir=weights_dir\\n",\n',
                '                ")\\n",\n',
                '                "tokenizer.pad_token = tokenizer.eos_token",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 9,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "def print_trainable_parameters(model) -> None:\\n",\n',
                '                \'    """Prints the number of trainable parameters in the model."""\\n\',\n',
                '                "    trainable_params = 0\\n",\n',
                '                "    all_param = 0\\n",\n',
                '                "    for _, param in model.named_parameters():\\n",\n',
                '                "        all_param += param.numel()\\n",\n',
                '                "        if param.requires_grad:\\n",\n',
                '                "            trainable_params += param.numel()\\n",\n',
                '                "\\n",\n',
                '                "    print(\\n",\n',
                "                '        f\"Trainable params num: {trainable_params:,} perzentage: {100 * trainable_params / all_param} || All params: {all_param:,}\"\\n',\n",
                '                "    )",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 10,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "Trainable params num: 262,410,240 perzentage: 6.993743675173274 || All params: 3,752,071,168\\n"\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": ["print_trainable_parameters(model)"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 11,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "model.gradient_checkpointing_enable()\\n",\n',
                '                "model = prepare_model_for_kbit_training(model)",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 13,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "Trainable params num: 42,520,576 perzentage: 1.1205573318192514 || All params: 3,794,591,744\\n"\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                "config = LoraConfig(\\n",\n',
                '                "    r=16,\\n",\n',
                '                "    lora_alpha=32,\\n",\n',
                '                "    target_modules=[\\n",\n',
                "                '        \"q_proj\",\\n',\n",
                "                '        \"k_proj\",\\n',\n",
                "                '        \"v_proj\",\\n',\n",
                "                '        \"o_proj\",\\n',\n",
                "                '        \"gate_proj\",\\n',\n",
                "                '        \"up_proj\",\\n',\n",
                "                '        \"down_proj\",\\n',\n",
                "                '        \"lm_head\",\\n',\n",
                '                "    ],\\n",\n',
                '                "    lora_dropout=0.05,\\n",\n',
                "                '    bias=\"none\",\\n',\n",
                "                '    task_type=\"CAUSAL_LM\",\\n',\n",
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "model = get_peft_model(model, config)\\n",\n',
                '                "print_trainable_parameters(model)",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "markdown",\n',
                '            "metadata": {},\n',
                '            "source": ["# Test original model\\n"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 34,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                \'prompt_template_large = """\\n\',\n',
                '                "<s>[INST]\\n",\n',
                '                "# TASK DESCRIPTION\\n",\n',
                '                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                "\\n",\n',
                '                "# JSON PARAMETER VALUES\\n",\n',
                '                \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                "- input_name: Name of the input parameter for the function.\\n",\n',
                '                "- input_value: Value for the function input parameter.\\n",\n',
                '                "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                "\\n",\n',
                '                "# FUNCTION MEANING\\n",\n',
                '                "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                "\\n",\n',
                '                "# RESPONSE FORMAT\\n",\n',
                '                "- Include only necessary functions from the query.\\n",\n',
                '                "- Maintain the order of functions as in the query.\\n",\n',
                '                "- Note any uncertainties in case of ambiguous queries.\\n",\n',
                '                "\\n",\n',
                '                "# ADDITIONAL GUIDANCE\\n",\n',
                '                "- Focus on verbs and technical terms to determine the function.\\n",\n',
                '                "- Indicate if a query falls outside the available functions.\\n",\n',
                '                "- Consider practical aspects of robotic operations.\\n",\n',
                '                \'- Treat "base" as the first robot "joint".\\n\',\n',
                '                "\\n",\n',
                '                "# JSON FORMAT\\n",\n',
                '                "{{\\n",\n',
                "                '    \"functions\": [\\n',\n",
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                "                '            \"inputs\": [\\n',\n",
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\\n\',\n',
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\\n\',\n',
                '                "            ]\\n",\n',
                '                "        }},\\n",\n',
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                '                \'            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\\n\',\n',
                '                "        }}\\n",\n',
                '                "    ]\\n",\n',
                '                "}}\\n",\n',
                '                "\\n",\n',
                '                "USER QUERY: {user_query}\\n",\n',
                '                "[/INST]\\n",\n',
                '                \'""".strip()\',\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 51,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                \'prompt_template_small = """\\n\',\n',
                '                "<s>[INST]\\n",\n',
                '                "# TASK DESCRIPTION\\n",\n',
                '                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                "\\n",\n',
                '                "# JSON PARAMETER VALUES\\n",\n',
                '                \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                "- input_name: Name of the input parameter for the function.\\n",\n',
                '                "- input_value: Value for the function input parameter.\\n",\n',
                '                "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                "\\n",\n',
                '                "# FUNCTION MEANING\\n",\n',
                '                "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                "\\n",\n',
                '                "# JSON FORMAT\\n",\n',
                '                "{{\\n",\n',
                "                '    \"functions\": [\\n',\n",
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                "                '            \"inputs\": [\\n',\n",
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\\n\',\n',
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\\n\',\n',
                '                "            ]\\n",\n',
                '                "        }},\\n",\n',
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                '                \'            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\\n\',\n',
                '                "        }}\\n",\n',
                '                "    ]\\n",\n',
                '                "}}\\n",\n',
                '                "\\n",\n',
                '                "USER QUERY: {user_query}\\n",\n',
                '                "[/INST]\\n",\n',
                '                \'""".strip()\',\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 75,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "generation_config = model.generation_config\\n",\n',
                '                "g   eneration_config.max_new_tokens = 200\\n",\n',
                '                "ge      neration_config.temperature = 0.7\\n",\n',
                '                "gen         eration_config.top_p = 0.7\\n",\n',
                '                "gen         eration_config.num_return_sequences = 1\\n",\n',
                '                "         generation_config.pad_token_id = tokenizer.eos_token_id\\n",\n',
                "                'gen         eration_config.eos_token_id = tokenizer.eos_token_id                     \"import os\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                        \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                              ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                              ,\\n",\n',
                "                \"                                          '',\\n\",\n",
                "                \"                '',\\n\",\n",
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                              \'\\"\\\\n\',\\n",\n',
                '                "                \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                                \'\\",\\\\n\',\\n",\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "                                                ,\\n",\n',
                '                "         ,\\n",\n',
                '                \'                                                                                 \\\'    """""\\\\n\\\',\\n\',\n',
                "                '                                                                                    \"\\\\n\",\\n',\n",
                '                "                        \'\\"\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                              ,\\n",\n',
                '                "                                                                                 ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                                                                                          \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                                    \'\\"none\\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                "                '                        \"\\\\n\",\\n',\n",
                '                "                        ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                                            \'\\"base\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                                                             ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                         \'\\\\\\"\',\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                "                '                                                                                                      \"\\\\n\",\\n',\n",
                '                "                                    \'\\"and\\", \',\\n",\n',
                "                '                                                                                          \"\\\\n\",\\n',\n",
                "                '                                                                                          \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                                        \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                        \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\',\\n",\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                "                '                        \\'\"name\": \",\\\\n\\',\\n',\n",
                '                "                                    ,\\n",\n',
                '                "               ,\\n",\n',
                '                "                                                      \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                        \'\\"pt\\"\\\\n\',\\n",\n',
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                "                \"                                                                                                                                          '',\\n\",\n",
                "                \"                                    '',\\n\",\n",
                '                "                  ,\\n",\n',
                '                "               ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                                          \'    \\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                '                "                \'\\"data\\",\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                  ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 76,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\'user_query = "Obtain the status of the robot\\\'s third joint"\'],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 77,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "[INST]\\n",\n',
                '                        "# TASK DESCRIPTION\\n",\n',
                '                        "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                        "\\n",\n',
                '                        "# JSON PARAMETER VALUES\\n",\n',
                '                        \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                        "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                        "- input_name: Name of the input parameter for the function.\\n",\n',
                '                        "- input_value: Value for the function input parameter.\\n",\n',
                '                        "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                        "\\n",\n',
                '                        "# FUNCTION MEANING\\n",\n',
                '                        "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                        "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                        "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                        "\\n",\n',
                '                        "# JSON FORMAT\\n",\n',
                '                        "{\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "<name>",\\n\',\n',
                "                        '            \"inputs\": [\\n',\n",
                '                        \'                {"name": "<name>", "value": "<value>", "unit": "<unit>"},\\n\',\n',
                '                        \'                {"name": "<name>", "value": "<value>", "unit": "<unit>"}\\n\',\n',
                '                        "            ]\\n",\n',
                '                        "        },\\n",\n',
                '                        "        {\\n",\n',
                '                        \'            "function_name": "<name>",\\n\',\n',
                '                        \'            "inputs": [{"name": "<name>", "value": "<value>", "unit": "<unit>"}]\\n\',\n',
                '                        "        }\\n",\n',
                '                        "    ]\\n",\n',
                '                        "}\\n",\n',
                '                        "\\n",\n',
                '                        "USER QUERY: Obtain the status of the robot\'s third joint\\n",\n',
                '                        "[/INST] {\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "get_joint_values",\\n\',\n',
                "                        '            \"inputs\": []\\n',\n",
                '                        "        }\\n",\n',
                '                        "    ]\\n",\n',
                '                        "}\\n",\n',
                '                        "\\n",\n',
                '                        "USER QUERY: Move the robot\'s end effector to position (200, 300, 400) and orientation (0.1, 0.2, 0.3, 0.4)\\n",\n',
                '                        "{\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "move_tcp",\\n\',\n',
                "                        '            \"inputs\": [\\n',\n",
                '                        \'                {"name": "x", "value": 200, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "y", "value": 300, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "z", "value": 400, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "q1",\\n\',\n',
                '                        "CPU times: user 8.04 s, sys: 0 ns, total: 8.04 s\\n",\n',
                '                        "Wall time: 8.04 s\\n",\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                "%%time\\n",\n',
                "                'device = \"cuda:0\"\\n',\n",
                '                "\\n",\n',
                '                "encoding = tokenizer(\\n",\n',
                "                '    prompt_template_small.format(user_query=user_query), return_tensors=\"pt\"\\n',\n",
                '                ").to(device)\\n",\n',
                '                "with torch.inference_mode():\\n",\n',
                '                "    outputs = model.generate(\\n",\n',
                '                "        input_ids=encoding.input_ids,\\n",\n',
                '                "        attention_mask=encoding.attention_mask,\\n",\n',
                '                "        generation_config=generation_config,\\n",\n',
                '                "    )\\n",\n',
                '                "\\n",\n',
                '                "print(tokenizer.decode(outputs[0], skip_special_tokens=True))",\n',
                "            ],\n",
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Prepare data\\n"]},\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 5,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "799088593edf4f5ca4a9d96197c77670",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Downloading data files:   0%|          | 0/1 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "1624861f54584c94a26e6959e1efd8ff",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "7f613b42d76c4638a525ff4ec75f91ec",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Generating train split: 0 examples [00:00, ? examples/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "ename": "NotImplementedError",\n',
                '                    "evalue": "Loading a dataset cached in a LocalFileSystem is not supported.",\n',
                '                    "output_type": "error",\n',
                '                    "traceback": [\n',
                '                        "\\u001b[0;31m---------------------------------------------------------------------------\\u001b[0m",\n',
                '                        "\\u001b[0;31mNotImplementedError\\u001b[0m                       Traceback (most recent call last)",\n',
                '                        \'Cell \\u001b[0;32mIn[5], line 1\\u001b[0m\\n\\u001b[0;32m----> 1\\u001b[0m dataset \\u001b[38;5;241m=\\u001b[39m \\u001b[43mload_dataset\\u001b[49m\\u001b[43m(\\u001b[49m\\n\\u001b[1;32m      2\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43mjson\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      3\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[43mdata_files\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43m../DATA/processed_dataset_2024-Jan-30_23-29-10.json\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      4\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[43mfield\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43mdata\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      5\\u001b[0m \\u001b[43m)\\u001b[49m\\n\',\n',
                '                        "File \\u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/load.py:2149\\u001b[0m, in \\u001b[0;36mload_dataset\\u001b[0;34m(path, name, data_dir, data_files, split, cache_dir, features, download_config, download_mode, verification_mode, ignore_verifications, keep_in_memory, save_infos, revision, token, use_auth_token, task, streaming, num_proc, storage_options, **config_kwargs)\\u001b[0m\\n\\u001b[1;32m   2145\\u001b[0m \\u001b[38;5;66;03m# Build dataset for splits\\u001b[39;00m\\n\\u001b[1;32m   2146\\u001b[0m keep_in_memory \\u001b[38;5;241m=\\u001b[39m (\\n\\u001b[1;32m   2147\\u001b[0m     keep_in_memory \\u001b[38;5;28;01mif\\u001b[39;00m keep_in_memory \\u001b[38;5;129;01mis\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m \\u001b[38;5;28;01mNone\\u001b[39;00m \\u001b[38;5;28;01melse\\u001b[39;00m is_small_dataset(builder_instance\\u001b[38;5;241m.\\u001b[39minfo\\u001b[38;5;241m.\\u001b[39mdataset_size)\\n\\u001b[1;32m   2148\\u001b[0m )\\n\\u001b[0;32m-> 2149\\u001b[0m ds \\u001b[38;5;241m=\\u001b[39m \\u001b[43mbuilder_instance\\u001b[49m\\u001b[38;5;241;43m.\\u001b[39;49m\\u001b[43mas_dataset\\u001b[49m\\u001b[43m(\\u001b[49m\\u001b[43msplit\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43msplit\\u001b[49m\\u001b[43m,\\u001b[49m\\u001b[43m \\u001b[49m\\u001b[43mverification_mode\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43mverification_mode\\u001b[49m\\u001b[43m,\\u001b[49m\\u001b[43m \\u001b[49m\\u001b[43min_memory\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43mkeep_in_memory\\u001b[49m\\u001b[43m)\\u001b[49m\\n\\u001b[1;32m   2150\\u001b[0m \\u001b[38;5;66;03m# Rename and cast features to match task schema\\u001b[39;00m\\n\\u001b[1;32m   2151\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m task \\u001b[38;5;129;01mis\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m \\u001b[38;5;28;01mNone\\u001b[39;00m:\\n\\u001b[1;32m   2152\\u001b[0m     \\u001b[38;5;66;03m# To avoid issuing the same warning twice\\u001b[39;00m\\n",\n',
                '                        \'File \\u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/builder.py:1173\\u001b[0m, in \\u001b[0;36mDatasetBuilder.as_dataset\\u001b[0;34m(self, split, run_post_process, verification_mode, ignore_verifications, in_memory)\\u001b[0m\\n\\u001b[1;32m   1171\\u001b[0m is_local \\u001b[38;5;241m=\\u001b[39m \\u001b[38;5;129;01mnot\\u001b[39;00m is_remote_filesystem(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_fs)\\n\\u001b[1;32m   1172\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m is_local:\\n\\u001b[0;32m-> 1173\\u001b[0m     \\u001b[38;5;28;01mraise\\u001b[39;00m \\u001b[38;5;167;01mNotImplementedError\\u001b[39;00m(\\u001b[38;5;124mf\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mLoading a dataset cached in a \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mtype\\u001b[39m(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_fs)\\u001b[38;5;241m.\\u001b[39m\\u001b[38;5;18m__name__\\u001b[39m\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m is not supported.\\u001b[39m\\u001b[38;5;124m"\\u001b[39m)\\n\\u001b[1;32m   1174\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m os\\u001b[38;5;241m.\\u001b[39mpath\\u001b[38;5;241m.\\u001b[39mexists(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_output_dir):\\n\\u001b[1;32m   1175\\u001b[0m     \\u001b[38;5;28;01mraise\\u001b[39;00m \\u001b[38;5;167;01mFileNotFoundError\\u001b[39;00m(\\n\\u001b[1;32m   1176\\u001b[0m         \\u001b[38;5;124mf\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mDataset \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39mdataset_name\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m: could not find data in \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_output_dir\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m. Please make sure to call \\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1177\\u001b[0m         \\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mbuilder.download_and_prepare(), or use \\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1178\\u001b[0m         \\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mdatasets.load_dataset() before trying to access the Dataset object.\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1179\\u001b[0m     )\\n\',\n',
                '                        "\\u001b[0;31mNotImplementedError\\u001b[0m: Loading a dataset cached in a LocalFileSystem is not supported.",\n',
                "                    ],\n",
                "                },\n",
                "            ],\n",
                '            "source": [\n',
                '                "dataset = load_dataset(\\n",\n',
                "                '    \"json\",\\n',\n",
                "                '    data_files=\"../DATA/processed_dataset_2024-Jan-30_23-29-10.json\",\\n',\n",
                "                '    field=\"data\",\\n',\n",
                '                ")",\n',
                "            ],\n",
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Fine-Tuning\\n"]},\n',
                '        {"cell_type": "markdown", "metadata": {}, "source": []},\n',
                "    ],\n",
                '    "metadata": {\n',
                '        "kernelspec": {\n',
                '            "display_name": "robot-whisperer-lfIYXQcw-py3.10",\n',
                '            "language": "python",\n',
                '            "name": "python3",\n',
                "        },\n",
                '        "language_info": {\n',
                '            "codemirror_mode": {"name": "ipython", "version": 3},\n',
                '            "file_extension": ".py",\n',
                '            "mimetype": "text/x-python",\n',
                '            "name": "python",\n',
                '            "nbconvert_exporter": "python",\n',
                '            "pygments_lexer": "ipython3",\n',
                '            "version": "3.10.13",\n',
                "        },\n",
                "    },\n",
                '    "nbformat": 4,\n',
                '    "nbformat_minor": 2,\n',
                "}",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["# Load model & Tokenizer\n"],
        },
        {
            "cell_type": "code",
            "execution_count": 5,
            "metadata": {},
            "outputs": [
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "9247f78f9c3949b1949207c721e3c698",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "bb47838913be4050892e98a3dff5e963",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "b1691fce8bf04f1eb054a8c82f173855",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "Downloading shards:   0%|          | 0/3 [00:00<?, ?it/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "c223b4be9917420ba66600995e0b9f67",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "11a5e85988c84a7783ce746323bb83b4",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "e09398773fc146ea8ecede8f329bd954",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "8a929f80d6394bc3a1090fbfa36484b7",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "9d21a25ab6e64231ac4593e3078eefed",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "223aed03a73a4ddc85ff82f099475564",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "f1041d3c6e86420bb9a017620b8d34ec",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "659f749ddc7345ddb7ab3968f34965d8",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "fc4e61e6baad4335b1c69dc0998837cf",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
            ],
            "source": [
                'MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"\n',
                'weights_dir = "./weights"\n',
                "\n",
                "bnb_config = BitsAndBytesConfig(\n",
                "    load_in_4bit=True,\n",
                "    bnb_4bit_use_double_quant=True,\n",
                '    bnb_4bit_quant_type="nf4",\n',
                "    bnb_4bit_compute_dtype=torch.bfloat16,\n",
                ")\n",
                "\n",
                "model = AutoModelForCausalLM.from_pretrained(\n",
                "    MODEL_NAME,\n",
                "    device_map=device,\n",
                "    trust_remote_code=True,\n",
                "    quantization_config=bnb_config,\n",
                "    cache_dir=weights_dir,\n",
                ")\n",
                "\n",
                "tokenizer = AutoTokenizer.from_pretrained(\n",
                "    MODEL_NAME, device=device, cache_dir=weights_dir\n",
                ")\n",
                "tokenizer.pad_token = tokenizer.eos_token",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 9,
            "metadata": {},
            "outputs": [],
            "source": [
                "def print_trainable_parameters(model) -> None:\n",
                '    """Prints the number of trainable parameters in the model."""\n',
                "    trainable_params = 0\n",
                "    all_param = 0\n",
                "    for _, param in model.named_parameters():\n",
                "        all_param += param.numel()\n",
                "        if param.requires_grad:\n",
                "            trainable_params += param.numel()\n",
                "\n",
                "    print(\n",
                '        f"Trainable params num: {trainable_params:,} perzentage: {100 * trainable_params / all_param} || All params: {all_param:,}"\n',
                "    )",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 10,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Trainable params num: 262,410,240 perzentage: 6.993743675173274 || All params: 3,752,071,168\n"
                    ],
                }
            ],
            "source": ["print_trainable_parameters(model)"],
        },
        {
            "cell_type": "code",
            "execution_count": 11,
            "metadata": {},
            "outputs": [],
            "source": [
                "model.gradient_checkpointing_enable()\n",
                "model = prepare_model_for_kbit_training(model)",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 13,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Trainable params num: 42,520,576 perzentage: 1.1205573318192514 || All params: 3,794,591,744\n"
                    ],
                }
            ],
            "source": [
                "config = LoraConfig(\n",
                "    r=16,\n",
                "    lora_alpha=32,\n",
                "    target_modules=[\n",
                '        "q_proj",\n',
                '        "k_proj",\n',
                '        "v_proj",\n',
                '        "o_proj",\n',
                '        "gate_proj",\n',
                '        "up_proj",\n',
                '        "down_proj",\n',
                '        "lm_head",\n',
                "    ],\n",
                "    lora_dropout=0.05,\n",
                '    bias="none",\n',
                '    task_type="CAUSAL_LM",\n',
                ")\n",
                "\n",
                "model = get_peft_model(model, config)\n",
                "print_trainable_parameters(model)",
            ],
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["# Test original model\n"],
        },
        {
            "cell_type": "code",
            "execution_count": 34,
            "metadata": {},
            "outputs": [],
            "source": [
                'prompt_template_large = """\n',
                "<s>[INST]\n",
                "# TASK DESCRIPTION\n",
                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\n",
                "\n",
                "# JSON PARAMETER VALUES\n",
                '- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\n',
                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\n",
                "- input_name: Name of the input parameter for the function.\n",
                "- input_value: Value for the function input parameter.\n",
                "- inputs: Array of input_name, input_value pairs, completing the function call.\n",
                "\n",
                "# FUNCTION MEANING\n",
                "- move_tcp: Moves robot's tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\n",
                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\n",
                "- get_joint_values: Retrieves current robot joint status. No input parameters.\n",
                "\n",
                "# RESPONSE FORMAT\n",
                "- Include only necessary functions from the query.\n",
                "- Maintain the order of functions as in the query.\n",
                "- Note any uncertainties in case of ambiguous queries.\n",
                "\n",
                "# ADDITIONAL GUIDANCE\n",
                "- Focus on verbs and technical terms to determine the function.\n",
                "- Indicate if a query falls outside the available functions.\n",
                "- Consider practical aspects of robotic operations.\n",
                '- Treat "base" as the first robot "joint".\n',
                "\n",
                "# JSON FORMAT\n",
                "{{\n",
                '    "functions": [\n',
                "        {{\n",
                '            "function_name": "<name>",\n',
                '            "inputs": [\n',
                '                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\n',
                '                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\n',
                "            ]\n",
                "        }},\n",
                "        {{\n",
                '            "function_name": "<name>",\n',
                '            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\n',
                "        }}\n",
                "    ]\n",
                "}}\n",
                "\n",
                "USER QUERY: {user_query}\n",
                "[/INST]\n",
                '""".strip()',
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 51,
            "metadata": {},
            "outputs": [],
            "source": [
                'prompt_template_small = """\n',
                "<s>[INST]\n",
                "# TASK DESCRIPTION\n",
                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\n",
                "\n",
                "# JSON PARAMETER VALUES\n",
                '- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\n',
                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\n",
                "- input_name: Name of the input parameter for the function.\n",
                "- input_value: Value for the function input parameter.\n",
                "- inputs: Array of input_name, input_value pairs, completing the function call.\n",
                "\n",
                "# FUNCTION MEANING\n",
                "- move_tcp: Moves robot's tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\n",
                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\n",
                "- get_joint_values: Retrieves current robot joint status. No input parameters.\n",
                "\n",
                "# JSON FORMAT\n",
                "{{\n",
                '    "functions": [\n',
                "        {{\n",
                '            "function_name": "<name>",\n',
                '            "inputs": [\n',
                '                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\n',
                '                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\n',
                "            ]\n",
                "        }},\n",
                "        {{\n",
                '            "function_name": "<name>",\n',
                '            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\n',
                "        }}\n",
                "    ]\n",
                "}}\n",
                "\n",
                "USER QUERY: {user_query}\n",
                "[/INST]\n",
                '""".strip()',
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 75,
            "metadata": {},
            "outputs": [],
            "source": [
                "{\n",
                '    "cells": [\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 2,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "import os\\n",\n',
                '                "import json\\n",\n',
                '                "from pprint import pprint\\n",\n',
                '                "\\n",\n',
                '                "import torch\\n",\n',
                '                "import torch.nn as nn\\n",\n',
                '                "\\n",\n',
                '                "from datasets import load_dataset, DatasetBuilder\\n",\n',
                '                "\\n",\n',
                '                "import transformers\\n",\n',
                '                "from transformers import (\\n",\n',
                '                "    AutoConfig,\\n",\n',
                '                "    AutoModelForCausalLM,\\n",\n',
                '                "    AutoTokenizer,\\n",\n',
                '                "    BitsAndBytesConfig,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "from peft import (\\n",\n',
                '                "    LoraConfig,\\n",\n',
                '                "    PeftConfig,\\n",\n',
                '                "    PeftModel,\\n",\n',
                '                "    get_peft_model,\\n",\n',
                '                "    prepare_model_for_kbit_training,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "from huggingface_hub import notebook_login",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 2,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "12826930970c474a8c7b0feac6339b90",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "VBox(children=(HTML(value=\'<center> <img\\\\nsrc=https://huggingface.co/front/assets/huggingface_logo-noborder.sv…"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                }\n",
                "            ],\n",
                '            "source": ["notebook_login()"],\n',
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Set running device\\n"]},\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 3,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": ["[INFO] Currently running device: cuda\\n"],\n',
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                \'device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\\n\',\n',
                '                \'p   rint(f"[INFO] Currently running device: {device}")                                                               "import os\\\\n",\\n\',\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                        \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                              ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                              ,\\n",\n',
                "                \"                                          '',\\n\",\n",
                "                \"                '',\\n\",\n",
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                              \'\\"\\\\n\',\\n",\n',
                '                "                \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                                \'\\",\\\\n\',\\n",\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "                                                ,\\n",\n',
                '                "         ,\\n",\n',
                '                \'                                                                                 \\\'    """""\\\\n\\\',\\n\',\n',
                "                '                                                                                    \"\\\\n\",\\n',\n",
                '                "                        \'\\"\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                              ,\\n",\n',
                '                "                                                                                 ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                                                                                          \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                                    \'\\"none\\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                "                '                        \"\\\\n\",\\n',\n",
                '                "                        ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                                            \'\\"base\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                                                             ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                         \'\\\\\\"\',\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                "                '                                                                                                      \"\\\\n\",\\n',\n",
                '                "                                    \'\\"and\\", \',\\n",\n',
                "                '                                                                                          \"\\\\n\",\\n',\n",
                "                '                                                                                          \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                                        \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                        \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\',\\n",\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                "                '                        \\'\"name\": \",\\\\n\\',\\n',\n",
                '                "                                    ,\\n",\n',
                '                "               ,\\n",\n',
                '                "                                                      \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                        \'\\"pt\\"\\\\n\',\\n",\n',
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                "                \"                                                                                                                                          '',\\n\",\n",
                "                \"                                    '',\\n\",\n",
                '                "                  ,\\n",\n',
                '                "               ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                                          \'    \\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                '                "                \'\\"data\\",\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                  ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "markdown",\n',
                '            "metadata": {},\n',
                '            "source": ["# Load model & Tokenizer\\n"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 5,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "9247f78f9c3949b1949207c721e3c698",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "bb47838913be4050892e98a3dff5e963",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "b1691fce8bf04f1eb054a8c82f173855",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Downloading shards:   0%|          | 0/3 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "c223b4be9917420ba66600995e0b9f67",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "11a5e85988c84a7783ce746323bb83b4",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "e09398773fc146ea8ecede8f329bd954",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "8a929f80d6394bc3a1090fbfa36484b7",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "9d21a25ab6e64231ac4593e3078eefed",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "223aed03a73a4ddc85ff82f099475564",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "f1041d3c6e86420bb9a017620b8d34ec",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "659f749ddc7345ddb7ab3968f34965d8",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "fc4e61e6baad4335b1c69dc0998837cf",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "            ],\n",
                '            "source": [\n',
                "                'MODEL_NAME = \"mistralai/Mistral-7B-Instruct-v0.2\"\\n',\n",
                "                'weights_dir = \"./weights\"\\n',\n",
                '                "\\n",\n',
                '                "bnb_config = BitsAndBytesConfig(\\n",\n',
                '                "    load_in_4bit=True,\\n",\n',
                '                "    bnb_4bit_use_double_quant=True,\\n",\n',
                "                '    bnb_4bit_quant_type=\"nf4\",\\n',\n",
                '                "    bnb_4bit_compute_dtype=torch.bfloat16,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "model = AutoModelForCausalLM.from_pretrained(\\n",\n',
                '                "    MODEL_NAME,\\n",\n',
                '                "    device_map=device,\\n",\n',
                '                "    trust_remote_code=True,\\n",\n',
                '                "    quantization_config=bnb_config,\\n",\n',
                '                "    cache_dir=weights_dir,\\n",\n',
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "tokenizer = AutoTokenizer.from_pretrained(\\n",\n',
                '                "    MODEL_NAME, device=device, cache_dir=weights_dir\\n",\n',
                '                ")\\n",\n',
                '                "tokenizer.pad_token = tokenizer.eos_token",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 9,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "def print_trainable_parameters(model) -> None:\\n",\n',
                '                \'    """Prints the number of trainable parameters in the model."""\\n\',\n',
                '                "    trainable_params = 0\\n",\n',
                '                "    all_param = 0\\n",\n',
                '                "    for _, param in model.named_parameters():\\n",\n',
                '                "        all_param += param.numel()\\n",\n',
                '                "        if param.requires_grad:\\n",\n',
                '                "            trainable_params += param.numel()\\n",\n',
                '                "\\n",\n',
                '                "    print(\\n",\n',
                "                '        f\"Trainable params num: {trainable_params:,} perzentage: {100 * trainable_params / all_param} || All params: {all_param:,}\"\\n',\n",
                '                "    )",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 10,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "Trainable params num: 262,410,240 perzentage: 6.993743675173274 || All params: 3,752,071,168\\n"\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": ["print_trainable_parameters(model)"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 11,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "model.gradient_checkpointing_enable()\\n",\n',
                '                "model = prepare_model_for_kbit_training(model)",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 13,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "Trainable params num: 42,520,576 perzentage: 1.1205573318192514 || All params: 3,794,591,744\\n"\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                "config = LoraConfig(\\n",\n',
                '                "    r=16,\\n",\n',
                '                "    lora_alpha=32,\\n",\n',
                '                "    target_modules=[\\n",\n',
                "                '        \"q_proj\",\\n',\n",
                "                '        \"k_proj\",\\n',\n",
                "                '        \"v_proj\",\\n',\n",
                "                '        \"o_proj\",\\n',\n",
                "                '        \"gate_proj\",\\n',\n",
                "                '        \"up_proj\",\\n',\n",
                "                '        \"down_proj\",\\n',\n",
                "                '        \"lm_head\",\\n',\n",
                '                "    ],\\n",\n',
                '                "    lora_dropout=0.05,\\n",\n',
                "                '    bias=\"none\",\\n',\n",
                "                '    task_type=\"CAUSAL_LM\",\\n',\n",
                '                ")\\n",\n',
                '                "\\n",\n',
                '                "model = get_peft_model(model, config)\\n",\n',
                '                "print_trainable_parameters(model)",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "markdown",\n',
                '            "metadata": {},\n',
                '            "source": ["# Test original model\\n"],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 34,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                \'prompt_template_large = """\\n\',\n',
                '                "<s>[INST]\\n",\n',
                '                "# TASK DESCRIPTION\\n",\n',
                '                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                "\\n",\n',
                '                "# JSON PARAMETER VALUES\\n",\n',
                '                \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                "- input_name: Name of the input parameter for the function.\\n",\n',
                '                "- input_value: Value for the function input parameter.\\n",\n',
                '                "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                "\\n",\n',
                '                "# FUNCTION MEANING\\n",\n',
                '                "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                "\\n",\n',
                '                "# RESPONSE FORMAT\\n",\n',
                '                "- Include only necessary functions from the query.\\n",\n',
                '                "- Maintain the order of functions as in the query.\\n",\n',
                '                "- Note any uncertainties in case of ambiguous queries.\\n",\n',
                '                "\\n",\n',
                '                "# ADDITIONAL GUIDANCE\\n",\n',
                '                "- Focus on verbs and technical terms to determine the function.\\n",\n',
                '                "- Indicate if a query falls outside the available functions.\\n",\n',
                '                "- Consider practical aspects of robotic operations.\\n",\n',
                '                \'- Treat "base" as the first robot "joint".\\n\',\n',
                '                "\\n",\n',
                '                "# JSON FORMAT\\n",\n',
                '                "{{\\n",\n',
                "                '    \"functions\": [\\n',\n",
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                "                '            \"inputs\": [\\n',\n",
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\\n\',\n',
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\\n\',\n',
                '                "            ]\\n",\n',
                '                "        }},\\n",\n',
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                '                \'            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\\n\',\n',
                '                "        }}\\n",\n',
                '                "    ]\\n",\n',
                '                "}}\\n",\n',
                '                "\\n",\n',
                '                "USER QUERY: {user_query}\\n",\n',
                '                "[/INST]\\n",\n',
                '                \'""".strip()\',\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 51,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                \'prompt_template_small = """\\n\',\n',
                '                "<s>[INST]\\n",\n',
                '                "# TASK DESCRIPTION\\n",\n',
                '                "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                "\\n",\n',
                '                "# JSON PARAMETER VALUES\\n",\n',
                '                \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                "- input_name: Name of the input parameter for the function.\\n",\n',
                '                "- input_value: Value for the function input parameter.\\n",\n',
                '                "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                "\\n",\n',
                '                "# FUNCTION MEANING\\n",\n',
                '                "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                "\\n",\n',
                '                "# JSON FORMAT\\n",\n',
                '                "{{\\n",\n',
                "                '    \"functions\": [\\n',\n",
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                "                '            \"inputs\": [\\n',\n",
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},\\n\',\n',
                '                \'                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}\\n\',\n',
                '                "            ]\\n",\n',
                '                "        }},\\n",\n',
                '                "        {{\\n",\n',
                '                \'            "function_name": "<name>",\\n\',\n',
                '                \'            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]\\n\',\n',
                '                "        }}\\n",\n',
                '                "    ]\\n",\n',
                '                "}}\\n",\n',
                '                "\\n",\n',
                '                "USER QUERY: {user_query}\\n",\n',
                '                "[/INST]\\n",\n',
                '                \'""".strip()\',\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 75,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\n',
                '                "generation_config = model.generation_config\\n",\n',
                '                "g   eneration_config.max_new_tokens = 200\\n",\n',
                '                "ge      neration_config.temperature = 0.7\\n",\n',
                '                "gen         eration_config.top_p = 0.7\\n",\n',
                '                "gen         eration_config.num_return_sequences = 1\\n",\n',
                '                "         generation_config.pad_token_id = tokenizer.eos_token_id\\n",\n',
                "                'gen         eration_config.eos_token_id = tokenizer.eos_token_id                     \"import os\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                        \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                              ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                              ,\\n",\n',
                "                \"                                          '',\\n\",\n",
                "                \"                '',\\n\",\n",
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                              \'\\"\\\\n\',\\n",\n',
                '                "                \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                                \'\\",\\\\n\',\\n",\n',
                "                '                                    \"\\\\n\",\\n',\n",
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "                                                ,\\n",\n',
                '                "         ,\\n",\n',
                '                \'                                                                                 \\\'    """""\\\\n\\\',\\n\',\n',
                "                '                                                                                    \"\\\\n\",\\n',\n",
                '                "                        \'\\"\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                              ,\\n",\n',
                '                "                                                                                 ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                '                "                                                               ,\\n",\n',
                '                "                                                                                          \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                \'        \\",\\\\n\',\\n",\n',
                '                "                                    \'\\"none\\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                "                '                        \"\\\\n\",\\n',\n",
                '                "                        ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                       ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                                            \'\\"base\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                                     \\'\"\"\"\\\\n\\',\\n',\n",
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                        \'\\"and\\", \',\\n",\n',
                "                '                                                            \"\\\\n\",\\n',\n",
                "                '                                                            \"\\\\n\",\\n',\n",
                '                "                                    \'    \\": [\\\\n\',\\n",\n',
                '                "                        \'            \\": \\",\\\\n\',\\n",\n',
                '                "                \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                \"\\\\n\",\\n',\n",
                '                "                                    \'\\"\\"\',\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                                                                                                             ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                                                         \'\\\\\\"\',\\n",\n',
                "                '                                                \"outputs\": [\\n',\n",
                '                \'                           "name": "stdout",\\n\',\n',
                "                '                                                                                                      \"\\\\n\",\\n',\n",
                '                "                                    \'\\"and\\", \',\\n",\n',
                "                '                                                                                          \"\\\\n\",\\n',\n",
                "                '                                                                                          \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                \'                        \\\'"name": ", ": ", "unit": \\\',\\n\',\n',
                '                "                                                                        \'            \\": \\",\\\\n\',\\n",\n',
                '                \'                        \\\'            "inputs"name": ", ": ", "unit": \\\',\\n\',\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\',\\n",\n',
                "                '                                                                        \"\\\\n\",\\n',\n",
                '                "                                                      \'    \\": [\\\\n\',\\n",\n',
                '                "                                    \'            \\": \\",\\\\n\',\\n",\n',
                '                "                        \'            \\"inputs\\": [\\\\n\',\\n",\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                '                \'                        \\\'"name": ", "unit": "mm\\\',\\n\',\n',
                "                '                        \\'\"name\": \",\\\\n\\',\\n',\n",
                '                "                                    ,\\n",\n',
                '                "               ,\\n",\n',
                '                "                                                      \'\\"\\\\n\',\\n",\n',
                "                '                \"\\\\n\",\\n',\n",
                '                "                        \'\\"pt\\"\\\\n\',\\n",\n',
                "                '                                                                                                \"\\\\n\",\\n',\n",
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                "                '                                                      \"outputs\": [\\n',\n",
                '                "                                                                                                            ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                '                "                                                                                                                        ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "                                             ,\\n",\n',
                "                \"                                                                                                                                          '',\\n\",\n",
                "                \"                                    '',\\n\",\n",
                '                "                  ,\\n",\n',
                '                "               ,\\n",\n',
                '                "            ,\\n",\n',
                '                "                                          \'    \\",\\\\n\',\\n",\n',
                '                "                \'\\",\\\\n\',\\n",\n',
                '                "                \'\\"data\\",\\\\n\',\\n",\n',
                '                "            ,\\n",\n',
                '                "         ,\\n",\n',
                '                "                  ,\\n",\n',
                '                "                                       ,\\n",\n',
                '                "                                                                           ,\\n",\n',
                '                "      ,\\n",\n',
                '                "         ,",\n',
                "            ],\n",
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 76,\n',
                '            "metadata": {},\n',
                '            "outputs": [],\n',
                '            "source": [\'user_query = "Obtain the status of the robot\\\'s third joint"\'],\n',
                "        },\n",
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 77,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "name": "stdout",\n',
                '                    "output_type": "stream",\n',
                '                    "text": [\n',
                '                        "[INST]\\n",\n',
                '                        "# TASK DESCRIPTION\\n",\n',
                '                        "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\\n",\n',
                '                        "\\n",\n',
                '                        "# JSON PARAMETER VALUES\\n",\n',
                '                        \'- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\\n\',\n',
                '                        "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\\n",\n',
                '                        "- input_name: Name of the input parameter for the function.\\n",\n',
                '                        "- input_value: Value for the function input parameter.\\n",\n',
                '                        "- inputs: Array of input_name, input_value pairs, completing the function call.\\n",\n',
                '                        "\\n",\n',
                '                        "# FUNCTION MEANING\\n",\n',
                '                        "- move_tcp: Moves robot\'s tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\\n",\n',
                '                        "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\\n",\n',
                '                        "- get_joint_values: Retrieves current robot joint status. No input parameters.\\n",\n',
                '                        "\\n",\n',
                '                        "# JSON FORMAT\\n",\n',
                '                        "{\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "<name>",\\n\',\n',
                "                        '            \"inputs\": [\\n',\n",
                '                        \'                {"name": "<name>", "value": "<value>", "unit": "<unit>"},\\n\',\n',
                '                        \'                {"name": "<name>", "value": "<value>", "unit": "<unit>"}\\n\',\n',
                '                        "            ]\\n",\n',
                '                        "        },\\n",\n',
                '                        "        {\\n",\n',
                '                        \'            "function_name": "<name>",\\n\',\n',
                '                        \'            "inputs": [{"name": "<name>", "value": "<value>", "unit": "<unit>"}]\\n\',\n',
                '                        "        }\\n",\n',
                '                        "    ]\\n",\n',
                '                        "}\\n",\n',
                '                        "\\n",\n',
                '                        "USER QUERY: Obtain the status of the robot\'s third joint\\n",\n',
                '                        "[/INST] {\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "get_joint_values",\\n\',\n',
                "                        '            \"inputs\": []\\n',\n",
                '                        "        }\\n",\n',
                '                        "    ]\\n",\n',
                '                        "}\\n",\n',
                '                        "\\n",\n',
                '                        "USER QUERY: Move the robot\'s end effector to position (200, 300, 400) and orientation (0.1, 0.2, 0.3, 0.4)\\n",\n',
                '                        "{\\n",\n',
                "                        '    \"functions\": [\\n',\n",
                '                        "        {\\n",\n',
                '                        \'            "function_name": "move_tcp",\\n\',\n',
                "                        '            \"inputs\": [\\n',\n",
                '                        \'                {"name": "x", "value": 200, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "y", "value": 300, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "z", "value": 400, "unit": "mm"},\\n\',\n',
                '                        \'                {"name": "q1",\\n\',\n',
                '                        "CPU times: user 8.04 s, sys: 0 ns, total: 8.04 s\\n",\n',
                '                        "Wall time: 8.04 s\\n",\n',
                "                    ],\n",
                "                }\n",
                "            ],\n",
                '            "source": [\n',
                '                "%%time\\n",\n',
                "                'device = \"cuda:0\"\\n',\n",
                '                "\\n",\n',
                '                "encoding = tokenizer(\\n",\n',
                "                '    prompt_template_small.format(user_query=user_query), return_tensors=\"pt\"\\n',\n",
                '                ").to(device)\\n",\n',
                '                "with torch.inference_mode():\\n",\n',
                '                "    outputs = model.generate(\\n",\n',
                '                "        input_ids=encoding.input_ids,\\n",\n',
                '                "        attention_mask=encoding.attention_mask,\\n",\n',
                '                "        generation_config=generation_config,\\n",\n',
                '                "    )\\n",\n',
                '                "\\n",\n',
                '                "print(tokenizer.decode(outputs[0], skip_special_tokens=True))",\n',
                "            ],\n",
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Prepare data\\n"]},\n',
                "        {\n",
                '            "cell_type": "code",\n',
                '            "execution_count": 5,\n',
                '            "metadata": {},\n',
                '            "outputs": [\n',
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "799088593edf4f5ca4a9d96197c77670",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Downloading data files:   0%|          | 0/1 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "1624861f54584c94a26e6959e1efd8ff",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "data": {\n',
                '                        "application/vnd.jupyter.widget-view+json": {\n',
                '                            "model_id": "7f613b42d76c4638a525ff4ec75f91ec",\n',
                '                            "version_major": 2,\n',
                '                            "version_minor": 0,\n',
                "                        },\n",
                '                        "text/plain": [\n',
                '                            "Generating train split: 0 examples [00:00, ? examples/s]"\n',
                "                        ],\n",
                "                    },\n",
                '                    "metadata": {},\n',
                '                    "output_type": "display_data",\n',
                "                },\n",
                "                {\n",
                '                    "ename": "NotImplementedError",\n',
                '                    "evalue": "Loading a dataset cached in a LocalFileSystem is not supported.",\n',
                '                    "output_type": "error",\n',
                '                    "traceback": [\n',
                '                        "\\u001b[0;31m---------------------------------------------------------------------------\\u001b[0m",\n',
                '                        "\\u001b[0;31mNotImplementedError\\u001b[0m                       Traceback (most recent call last)",\n',
                '                        \'Cell \\u001b[0;32mIn[5], line 1\\u001b[0m\\n\\u001b[0;32m----> 1\\u001b[0m dataset \\u001b[38;5;241m=\\u001b[39m \\u001b[43mload_dataset\\u001b[49m\\u001b[43m(\\u001b[49m\\n\\u001b[1;32m      2\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43mjson\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      3\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[43mdata_files\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43m../DATA/processed_dataset_2024-Jan-30_23-29-10.json\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      4\\u001b[0m \\u001b[43m    \\u001b[49m\\u001b[43mfield\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[38;5;124;43mdata\\u001b[39;49m\\u001b[38;5;124;43m"\\u001b[39;49m\\u001b[43m,\\u001b[49m\\n\\u001b[1;32m      5\\u001b[0m \\u001b[43m)\\u001b[49m\\n\',\n',
                '                        "File \\u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/load.py:2149\\u001b[0m, in \\u001b[0;36mload_dataset\\u001b[0;34m(path, name, data_dir, data_files, split, cache_dir, features, download_config, download_mode, verification_mode, ignore_verifications, keep_in_memory, save_infos, revision, token, use_auth_token, task, streaming, num_proc, storage_options, **config_kwargs)\\u001b[0m\\n\\u001b[1;32m   2145\\u001b[0m \\u001b[38;5;66;03m# Build dataset for splits\\u001b[39;00m\\n\\u001b[1;32m   2146\\u001b[0m keep_in_memory \\u001b[38;5;241m=\\u001b[39m (\\n\\u001b[1;32m   2147\\u001b[0m     keep_in_memory \\u001b[38;5;28;01mif\\u001b[39;00m keep_in_memory \\u001b[38;5;129;01mis\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m \\u001b[38;5;28;01mNone\\u001b[39;00m \\u001b[38;5;28;01melse\\u001b[39;00m is_small_dataset(builder_instance\\u001b[38;5;241m.\\u001b[39minfo\\u001b[38;5;241m.\\u001b[39mdataset_size)\\n\\u001b[1;32m   2148\\u001b[0m )\\n\\u001b[0;32m-> 2149\\u001b[0m ds \\u001b[38;5;241m=\\u001b[39m \\u001b[43mbuilder_instance\\u001b[49m\\u001b[38;5;241;43m.\\u001b[39;49m\\u001b[43mas_dataset\\u001b[49m\\u001b[43m(\\u001b[49m\\u001b[43msplit\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43msplit\\u001b[49m\\u001b[43m,\\u001b[49m\\u001b[43m \\u001b[49m\\u001b[43mverification_mode\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43mverification_mode\\u001b[49m\\u001b[43m,\\u001b[49m\\u001b[43m \\u001b[49m\\u001b[43min_memory\\u001b[49m\\u001b[38;5;241;43m=\\u001b[39;49m\\u001b[43mkeep_in_memory\\u001b[49m\\u001b[43m)\\u001b[49m\\n\\u001b[1;32m   2150\\u001b[0m \\u001b[38;5;66;03m# Rename and cast features to match task schema\\u001b[39;00m\\n\\u001b[1;32m   2151\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m task \\u001b[38;5;129;01mis\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m \\u001b[38;5;28;01mNone\\u001b[39;00m:\\n\\u001b[1;32m   2152\\u001b[0m     \\u001b[38;5;66;03m# To avoid issuing the same warning twice\\u001b[39;00m\\n",\n',
                '                        \'File \\u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/builder.py:1173\\u001b[0m, in \\u001b[0;36mDatasetBuilder.as_dataset\\u001b[0;34m(self, split, run_post_process, verification_mode, ignore_verifications, in_memory)\\u001b[0m\\n\\u001b[1;32m   1171\\u001b[0m is_local \\u001b[38;5;241m=\\u001b[39m \\u001b[38;5;129;01mnot\\u001b[39;00m is_remote_filesystem(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_fs)\\n\\u001b[1;32m   1172\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m is_local:\\n\\u001b[0;32m-> 1173\\u001b[0m     \\u001b[38;5;28;01mraise\\u001b[39;00m \\u001b[38;5;167;01mNotImplementedError\\u001b[39;00m(\\u001b[38;5;124mf\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mLoading a dataset cached in a \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mtype\\u001b[39m(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_fs)\\u001b[38;5;241m.\\u001b[39m\\u001b[38;5;18m__name__\\u001b[39m\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m is not supported.\\u001b[39m\\u001b[38;5;124m"\\u001b[39m)\\n\\u001b[1;32m   1174\\u001b[0m \\u001b[38;5;28;01mif\\u001b[39;00m \\u001b[38;5;129;01mnot\\u001b[39;00m os\\u001b[38;5;241m.\\u001b[39mpath\\u001b[38;5;241m.\\u001b[39mexists(\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_output_dir):\\n\\u001b[1;32m   1175\\u001b[0m     \\u001b[38;5;28;01mraise\\u001b[39;00m \\u001b[38;5;167;01mFileNotFoundError\\u001b[39;00m(\\n\\u001b[1;32m   1176\\u001b[0m         \\u001b[38;5;124mf\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mDataset \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39mdataset_name\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m: could not find data in \\u001b[39m\\u001b[38;5;132;01m{\\u001b[39;00m\\u001b[38;5;28mself\\u001b[39m\\u001b[38;5;241m.\\u001b[39m_output_dir\\u001b[38;5;132;01m}\\u001b[39;00m\\u001b[38;5;124m. Please make sure to call \\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1177\\u001b[0m         \\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mbuilder.download_and_prepare(), or use \\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1178\\u001b[0m         \\u001b[38;5;124m"\\u001b[39m\\u001b[38;5;124mdatasets.load_dataset() before trying to access the Dataset object.\\u001b[39m\\u001b[38;5;124m"\\u001b[39m\\n\\u001b[1;32m   1179\\u001b[0m     )\\n\',\n',
                '                        "\\u001b[0;31mNotImplementedError\\u001b[0m: Loading a dataset cached in a LocalFileSystem is not supported.",\n',
                "                    ],\n",
                "                },\n",
                "            ],\n",
                '            "source": [\n',
                '                "dataset = load_dataset(\\n",\n',
                "                '    \"json\",\\n',\n",
                "                '    data_files=\"../DATA/processed_dataset_2024-Jan-30_23-29-10.json\",\\n',\n",
                "                '    field=\"data\",\\n',\n",
                '                ")",\n',
                "            ],\n",
                "        },\n",
                '        {"cell_type": "markdown", "metadata": {}, "source": ["# Fine-Tuning\\n"]},\n',
                '        {"cell_type": "markdown", "metadata": {}, "source": []},\n',
                "    ],\n",
                '    "metadata": {\n',
                '        "kernelspec": {\n',
                '            "display_name": "robot-whisperer-lfIYXQcw-py3.10",\n',
                '            "language": "python",\n',
                '            "name": "python3",\n',
                "        },\n",
                '        "language_info": {\n',
                '            "codemirror_mode": {"name": "ipython", "version": 3},\n',
                '            "file_extension": ".py",\n',
                '            "mimetype": "text/x-python",\n',
                '            "name": "python",\n',
                '            "nbconvert_exporter": "python",\n',
                '            "pygments_lexer": "ipython3",\n',
                '            "version": "3.10.13",\n',
                "        },\n",
                "    },\n",
                '    "nbformat": 4,\n',
                '    "nbformat_minor": 2,\n',
                "}",
            ],
        },
        {
            "cell_type": "code",
            "execution_count": 76,
            "metadata": {},
            "outputs": [],
            "source": ['user_query = "Obtain the status of the robot\'s third joint"'],
        },
        {
            "cell_type": "code",
            "execution_count": 77,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "[INST]\n",
                        "# TASK DESCRIPTION\n",
                        "Translate user queries about industrial robotic operations into JSON outputs for specific function calls.\n",
                        "\n",
                        "# JSON PARAMETER VALUES\n",
                        '- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".\n',
                        "- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.\n",
                        "- input_name: Name of the input parameter for the function.\n",
                        "- input_value: Value for the function input parameter.\n",
                        "- inputs: Array of input_name, input_value pairs, completing the function call.\n",
                        "\n",
                        "# FUNCTION MEANING\n",
                        "- move_tcp: Moves robot's tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).\n",
                        "- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).\n",
                        "- get_joint_values: Retrieves current robot joint status. No input parameters.\n",
                        "\n",
                        "# JSON FORMAT\n",
                        "{\n",
                        '    "functions": [\n',
                        "        {\n",
                        '            "function_name": "<name>",\n',
                        '            "inputs": [\n',
                        '                {"name": "<name>", "value": "<value>", "unit": "<unit>"},\n',
                        '                {"name": "<name>", "value": "<value>", "unit": "<unit>"}\n',
                        "            ]\n",
                        "        },\n",
                        "        {\n",
                        '            "function_name": "<name>",\n',
                        '            "inputs": [{"name": "<name>", "value": "<value>", "unit": "<unit>"}]\n',
                        "        }\n",
                        "    ]\n",
                        "}\n",
                        "\n",
                        "USER QUERY: Obtain the status of the robot's third joint\n",
                        "[/INST] {\n",
                        '    "functions": [\n',
                        "        {\n",
                        '            "function_name": "get_joint_values",\n',
                        '            "inputs": []\n',
                        "        }\n",
                        "    ]\n",
                        "}\n",
                        "\n",
                        "USER QUERY: Move the robot's end effector to position (200, 300, 400) and orientation (0.1, 0.2, 0.3, 0.4)\n",
                        "{\n",
                        '    "functions": [\n',
                        "        {\n",
                        '            "function_name": "move_tcp",\n',
                        '            "inputs": [\n',
                        '                {"name": "x", "value": 200, "unit": "mm"},\n',
                        '                {"name": "y", "value": 300, "unit": "mm"},\n',
                        '                {"name": "z", "value": 400, "unit": "mm"},\n',
                        '                {"name": "q1",\n',
                        "CPU times: user 8.04 s, sys: 0 ns, total: 8.04 s\n",
                        "Wall time: 8.04 s\n",
                    ],
                }
            ],
            "source": [
                "%%time\n",
                'device = "cuda:0"\n',
                "\n",
                "encoding = tokenizer(\n",
                '    prompt_template_small.format(user_query=user_query), return_tensors="pt"\n',
                ").to(device)\n",
                "with torch.inference_mode():\n",
                "    outputs = model.generate(\n",
                "        input_ids=encoding.input_ids,\n",
                "        attention_mask=encoding.attention_mask,\n",
                "        generation_config=generation_config,\n",
                "    )\n",
                "\n",
                "print(tokenizer.decode(outputs[0], skip_special_tokens=True))",
            ],
        },
        {"cell_type": "markdown", "metadata": {}, "source": ["# Prepare data\n"]},
        {
            "cell_type": "code",
            "execution_count": 5,
            "metadata": {},
            "outputs": [
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "799088593edf4f5ca4a9d96197c77670",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "Downloading data files:   0%|          | 0/1 [00:00<?, ?it/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "1624861f54584c94a26e6959e1efd8ff",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "data": {
                        "application/vnd.jupyter.widget-view+json": {
                            "model_id": "7f613b42d76c4638a525ff4ec75f91ec",
                            "version_major": 2,
                            "version_minor": 0,
                        },
                        "text/plain": [
                            "Generating train split: 0 examples [00:00, ? examples/s]"
                        ],
                    },
                    "metadata": {},
                    "output_type": "display_data",
                },
                {
                    "ename": "NotImplementedError",
                    "evalue": "Loading a dataset cached in a LocalFileSystem is not supported.",
                    "output_type": "error",
                    "traceback": [
                        "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
                        "\u001b[0;31mNotImplementedError\u001b[0m                       Traceback (most recent call last)",
                        'Cell \u001b[0;32mIn[5], line 1\u001b[0m\n\u001b[0;32m----> 1\u001b[0m dataset \u001b[38;5;241m=\u001b[39m \u001b[43mload_dataset\u001b[49m\u001b[43m(\u001b[49m\n\u001b[1;32m      2\u001b[0m \u001b[43m    \u001b[49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[38;5;124;43mjson\u001b[39;49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[43m,\u001b[49m\n\u001b[1;32m      3\u001b[0m \u001b[43m    \u001b[49m\u001b[43mdata_files\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[38;5;124;43m../DATA/processed_dataset_2024-Jan-30_23-29-10.json\u001b[39;49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[43m,\u001b[49m\n\u001b[1;32m      4\u001b[0m \u001b[43m    \u001b[49m\u001b[43mfield\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[38;5;124;43mdata\u001b[39;49m\u001b[38;5;124;43m"\u001b[39;49m\u001b[43m,\u001b[49m\n\u001b[1;32m      5\u001b[0m \u001b[43m)\u001b[49m\n',
                        "File \u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/load.py:2149\u001b[0m, in \u001b[0;36mload_dataset\u001b[0;34m(path, name, data_dir, data_files, split, cache_dir, features, download_config, download_mode, verification_mode, ignore_verifications, keep_in_memory, save_infos, revision, token, use_auth_token, task, streaming, num_proc, storage_options, **config_kwargs)\u001b[0m\n\u001b[1;32m   2145\u001b[0m \u001b[38;5;66;03m# Build dataset for splits\u001b[39;00m\n\u001b[1;32m   2146\u001b[0m keep_in_memory \u001b[38;5;241m=\u001b[39m (\n\u001b[1;32m   2147\u001b[0m     keep_in_memory \u001b[38;5;28;01mif\u001b[39;00m keep_in_memory \u001b[38;5;129;01mis\u001b[39;00m \u001b[38;5;129;01mnot\u001b[39;00m \u001b[38;5;28;01mNone\u001b[39;00m \u001b[38;5;28;01melse\u001b[39;00m is_small_dataset(builder_instance\u001b[38;5;241m.\u001b[39minfo\u001b[38;5;241m.\u001b[39mdataset_size)\n\u001b[1;32m   2148\u001b[0m )\n\u001b[0;32m-> 2149\u001b[0m ds \u001b[38;5;241m=\u001b[39m \u001b[43mbuilder_instance\u001b[49m\u001b[38;5;241;43m.\u001b[39;49m\u001b[43mas_dataset\u001b[49m\u001b[43m(\u001b[49m\u001b[43msplit\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43msplit\u001b[49m\u001b[43m,\u001b[49m\u001b[43m \u001b[49m\u001b[43mverification_mode\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mverification_mode\u001b[49m\u001b[43m,\u001b[49m\u001b[43m \u001b[49m\u001b[43min_memory\u001b[49m\u001b[38;5;241;43m=\u001b[39;49m\u001b[43mkeep_in_memory\u001b[49m\u001b[43m)\u001b[49m\n\u001b[1;32m   2150\u001b[0m \u001b[38;5;66;03m# Rename and cast features to match task schema\u001b[39;00m\n\u001b[1;32m   2151\u001b[0m \u001b[38;5;28;01mif\u001b[39;00m task \u001b[38;5;129;01mis\u001b[39;00m \u001b[38;5;129;01mnot\u001b[39;00m \u001b[38;5;28;01mNone\u001b[39;00m:\n\u001b[1;32m   2152\u001b[0m     \u001b[38;5;66;03m# To avoid issuing the same warning twice\u001b[39;00m\n",
                        'File \u001b[0;32m~/.cache/pypoetry/virtualenvs/robot-whisperer-lfIYXQcw-py3.10/lib/python3.10/site-packages/datasets/builder.py:1173\u001b[0m, in \u001b[0;36mDatasetBuilder.as_dataset\u001b[0;34m(self, split, run_post_process, verification_mode, ignore_verifications, in_memory)\u001b[0m\n\u001b[1;32m   1171\u001b[0m is_local \u001b[38;5;241m=\u001b[39m \u001b[38;5;129;01mnot\u001b[39;00m is_remote_filesystem(\u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39m_fs)\n\u001b[1;32m   1172\u001b[0m \u001b[38;5;28;01mif\u001b[39;00m \u001b[38;5;129;01mnot\u001b[39;00m is_local:\n\u001b[0;32m-> 1173\u001b[0m     \u001b[38;5;28;01mraise\u001b[39;00m \u001b[38;5;167;01mNotImplementedError\u001b[39;00m(\u001b[38;5;124mf\u001b[39m\u001b[38;5;124m"\u001b[39m\u001b[38;5;124mLoading a dataset cached in a \u001b[39m\u001b[38;5;132;01m{\u001b[39;00m\u001b[38;5;28mtype\u001b[39m(\u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39m_fs)\u001b[38;5;241m.\u001b[39m\u001b[38;5;18m__name__\u001b[39m\u001b[38;5;132;01m}\u001b[39;00m\u001b[38;5;124m is not supported.\u001b[39m\u001b[38;5;124m"\u001b[39m)\n\u001b[1;32m   1174\u001b[0m \u001b[38;5;28;01mif\u001b[39;00m \u001b[38;5;129;01mnot\u001b[39;00m os\u001b[38;5;241m.\u001b[39mpath\u001b[38;5;241m.\u001b[39mexists(\u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39m_output_dir):\n\u001b[1;32m   1175\u001b[0m     \u001b[38;5;28;01mraise\u001b[39;00m \u001b[38;5;167;01mFileNotFoundError\u001b[39;00m(\n\u001b[1;32m   1176\u001b[0m         \u001b[38;5;124mf\u001b[39m\u001b[38;5;124m"\u001b[39m\u001b[38;5;124mDataset \u001b[39m\u001b[38;5;132;01m{\u001b[39;00m\u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39mdataset_name\u001b[38;5;132;01m}\u001b[39;00m\u001b[38;5;124m: could not find data in \u001b[39m\u001b[38;5;132;01m{\u001b[39;00m\u001b[38;5;28mself\u001b[39m\u001b[38;5;241m.\u001b[39m_output_dir\u001b[38;5;132;01m}\u001b[39;00m\u001b[38;5;124m. Please make sure to call \u001b[39m\u001b[38;5;124m"\u001b[39m\n\u001b[1;32m   1177\u001b[0m         \u001b[38;5;124m"\u001b[39m\u001b[38;5;124mbuilder.download_and_prepare(), or use \u001b[39m\u001b[38;5;124m"\u001b[39m\n\u001b[1;32m   1178\u001b[0m         \u001b[38;5;124m"\u001b[39m\u001b[38;5;124mdatasets.load_dataset() before trying to access the Dataset object.\u001b[39m\u001b[38;5;124m"\u001b[39m\n\u001b[1;32m   1179\u001b[0m     )\n',
                        "\u001b[0;31mNotImplementedError\u001b[0m: Loading a dataset cached in a LocalFileSystem is not supported.",
                    ],
                },
            ],
            "source": [
                "dataset = load_dataset(\n",
                '    "json",\n',
                '    data_files="../DATA/processed_dataset_2024-Jan-30_23-29-10.json",\n',
                '    field="data",\n',
                ")",
            ],
        },
        {"cell_type": "markdown", "metadata": {}, "source": ["# Fine-Tuning\n"]},
        {"cell_type": "markdown", "metadata": {}, "source": []},
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "robot-whisperer-lfIYXQcw-py3.10",
            "language": "python",
            "name": "python3",
        },
        "language_info": {
            "codemirror_mode": {"name": "ipython", "version": 3},
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.10.13",
        },
    },
    "nbformat": 4,
    "nbformat_minor": 2,
}

In [76]:
user_query = "Obtain the status of the robot's third joint"

In [77]:
%%time
device = "cuda:0"

encoding = tokenizer(
    prompt_template_small.format(user_query=user_query), return_tensors="pt"
).to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

[INST]
# TASK DESCRIPTION
Translate user queries about industrial robotic operations into JSON outputs for specific function calls.

# JSON PARAMETER VALUES
- functions: Array of function calls from the USER QUERY, identified by keywords like "and", "then".
- function_name: The required robot function, one of: move_tcp, move_joint, get_joint_values.
- input_name: Name of the input parameter for the function.
- input_value: Value for the function input parameter.
- inputs: Array of input_name, input_value pairs, completing the function call.

# FUNCTION MEANING
- move_tcp: Moves robot's tool center point. Inputs: x, y, z (position in mm), q1, q2, q3, q4 (quaternion values for orientation).
- move_joint: Rotates/moves a robot joint. Inputs: joint (joint index), angle (rotation in radians).
- get_joint_values: Retrieves current robot joint status. No input parameters.

# JSON FORMAT
{
    "functions": [
        {
            "function_name": "<name>",
            "inputs": [
             

# Prepare data


In [5]:
dataset = load_dataset(
    "json",
    data_files="../DATA/processed_dataset_2024-Jan-30_23-29-10.json",
    field="data",
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

# Fine-Tuning
